In [1]:
!pip install pysurvival

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 5.0 MB/s 
  Created wheel for pysurvival: filename=pysurvival-0.1.2-cp37-cp37m-linux_x86_64.whl size=3773556 sha256=7558d49c9cddd954f4e302ef57c4223f19c983ccd4cccf2e3384b868d0b9b4ee
  Stored in directory: /root/.cache/pip/wheels/1a/63/e2/32273d765a4e2f4ccac69c8adf97425ca80bab5d0c8447f120
  Created wheel for progressbar: filename=progressbar-2.5-py3-none-any.whl size=12082 sha256=a0fe0df380cbf1dad21ac097622dd7a4ec938e4ef7ca6574439fef15f7bbc0fe
  Stored in directory: /root/.cache/pip/wheels/f0/fd/1f/3e35ed57e94cd8ced38dd46771f1f0f94f65fec548659ed855
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1310 sha256=43b4a709e5e4bd99715430f03e28358a2ab3ec4767b774ba33ee392e7adb9d9e
  Stored in directory: /root/.cache/pip/wheels/46/ef/c3/157e41f5ee1372d1be90b09f74f82b10e391eaacca8f22d33e
Successfully built pysurvival progressbar

# Original

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from pysurvival.models.svm import LinearSVMModel
from pysurvival.models.simulations import SimulationModel
from pysurvival.utils.metrics import concordance_index
from sklearn.model_selection import train_test_split
from scipy.stats.stats import pearsonr

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. train_data _73.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [ ]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
data_train = new_train_df
data_test = new_test_df

In [ ]:
sim = SimulationModel( survival_distribution = 'exponential',
                       risk_type = 'linear',
                       censored_parameter = 1,
                       alpha = 3)

In [ ]:
features = [f for f in data_train.columns if f not in ['a_year_time', 'a_year_event']]

time_column = 'a_year_time'
event_column = 'a_year_event'

In [ ]:
X_train, X_test = data_train[features], data_test[features]
T_train, T_test = data_train['a_year_time'].values, data_test['a_year_time'].values
E_train, E_test = data_train['a_year_event'].values, data_test['a_year_event'].values

In [ ]:
from sklearn.preprocessing import StandardScaler

X_scaler = StandardScaler().fit(X_train)
X_train = X_scaler.transform(X_train)
X_test = X_scaler.transform(X_test)

In [ ]:
svm_model = LinearSVMModel()
svm_model.fit(X_train, T_train, E_train, init_method='he_uniform',
    with_bias = True, lr = 0.5,  tol = 1e-3,  l2_reg = 1e-3)

Performing Newton-Raphson optimization: 
 * Iteration #1 - Loss = 16927.064 - ||grad||^2 = 31518.89120 - ||diff_w|| = 14.60947
 * Iteration #2 - Loss = 4326.144 - ||grad||^2 = 16030.91540 - ||diff_w|| = 6.54366
 * Iteration #3 - Loss = 1361.492 - ||grad||^2 = 9014.43907 - ||diff_w|| = 3.52660
 * Iteration #4 - Loss = 456.777 - ||grad||^2 = 4960.26516 - ||diff_w|| = 1.74736
 * Iteration #5 - Loss = 207.525 - ||grad||^2 = 2463.37939 - ||diff_w|| = 0.69887
 * Iteration #6 - Loss = 161.870 - ||grad||^2 = 1124.99541 - ||diff_w|| = 0.33177
 * Iteration #7 - Loss = 156.809 - ||grad||^2 = 433.23240 - ||diff_w|| = 0.14839
 * Iteration #8 - Loss = 157.050 - ||grad||^2 = 157.19909 - ||diff_w|| = 0.06335
 * Iteration #9 - Loss = 157.266 - ||grad||^2 = 66.16266 - ||diff_w|| = 0.02839
 * Iteration #10 - Loss = 157.348 - ||grad||^2 = 29.36968 - ||diff_w|| = 0.01352
 * Iteration #11 - Loss = 157.359 - ||grad||^2 = 13.69255 - ||diff_w|| = 0.00661
 * Iteration #12 - Loss = 157.366 - ||grad||^2 = 6.53497

LinearSVMModel

In [ ]:
c_index = concordance_index(svm_model, X_test, T_test, E_test) #0.93
print('C-index: {:f}'.format(c_index))

C-index: 0.836011


In [ ]:
from pysurvival.utils import save_model

In [ ]:
save_model(svm_model,'/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/SSVM/SSVM_new(onehot)_original3.zip')

Saving the model to disk as /content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/SSVM/SSVM_new(onehot)_original3.zip


/usr/local/lib/python3.7/dist-packages/pysurvival/utils/__init__.py:132: FutureWarning: 'pyarrow.serialize' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  model.save(path_file)


# tgan-imbal-5000

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from pysurvival.models.svm import LinearSVMModel
from pysurvival.models.simulations import SimulationModel
from pysurvival.utils.metrics import concordance_index
from sklearn.model_selection import train_test_split
from scipy.stats.stats import pearsonr

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/TGAN-imbal-AR12/TGAN-syn-AR-one-of-all-5000.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [ ]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
data_train = new_train_df
data_test = new_test_df

In [ ]:
sim = SimulationModel( survival_distribution = 'exponential',
                       risk_type = 'linear',
                       censored_parameter = 1,
                       alpha = 3)

In [ ]:
features = [f for f in data_train.columns if f not in ['a_year_time', 'a_year_event']]

time_column = 'a_year_time'
event_column = 'a_year_event'

In [ ]:
X_train, X_test = data_train[features], data_test[features]
T_train, T_test = data_train['a_year_time'].values, data_test['a_year_time'].values
E_train, E_test = data_train['a_year_event'].values, data_test['a_year_event'].values

In [ ]:
from sklearn.preprocessing import StandardScaler

X_scaler = StandardScaler().fit(X_train)
X_train = X_scaler.transform(X_train)
X_test = X_scaler.transform(X_test)

In [ ]:
svm_model = LinearSVMModel()
svm_model.fit(X_train, T_train, E_train, init_method='he_uniform',
    with_bias = True, lr = 0.5,  tol = 1e-3,  l2_reg = 1e-3)

Performing Newton-Raphson optimization: 
 * Iteration #1 - Loss = 366513.137 - ||grad||^2 = 623268.55014 - ||diff_w|| = 16.81042
 * Iteration #2 - Loss = 92486.779 - ||grad||^2 = 315674.26546 - ||diff_w|| = 7.45467
 * Iteration #3 - Loss = 28304.694 - ||grad||^2 = 176614.77139 - ||diff_w|| = 4.04414
 * Iteration #4 - Loss = 8737.405 - ||grad||^2 = 98273.42776 - ||diff_w|| = 2.12753
 * Iteration #5 - Loss = 3014.306 - ||grad||^2 = 52666.79000 - ||diff_w|| = 1.00965
 * Iteration #6 - Loss = 1563.634 - ||grad||^2 = 27000.76318 - ||diff_w|| = 0.51704
 * Iteration #7 - Loss = 1345.118 - ||grad||^2 = 10080.58384 - ||diff_w|| = 0.27145
 * Iteration #8 - Loss = 1389.702 - ||grad||^2 = 2090.70360 - ||diff_w|| = 0.11440
 * Iteration #9 - Loss = 1394.102 - ||grad||^2 = 496.17070 - ||diff_w|| = 0.04188
 * Iteration #10 - Loss = 1394.035 - ||grad||^2 = 139.37463 - ||diff_w|| = 0.01671
 * Iteration #11 - Loss = 1394.124 - ||grad||^2 = 41.99066 - ||diff_w|| = 0.00727
 * Iteration #12 - Loss = 1394.13

LinearSVMModel

In [ ]:
c_index = concordance_index(svm_model, X_test, T_test, E_test) #0.93
print('C-index: {:f}'.format(c_index))

C-index: 0.807182


In [ ]:
from pysurvival.utils import save_model

In [ ]:
save_model(svm_model,'/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/SSVM/SSVM_new(onehot)_tgan-imbal-5000.zip')

Saving the model to disk as /content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/SSVM/SSVM_new(onehot)_tgan-imbal-5000.zip


/usr/local/lib/python3.7/dist-packages/pysurvival/utils/__init__.py:132: FutureWarning: 'pyarrow.serialize' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  model.save(path_file)


# tgan-imbal-10000

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from pysurvival.models.svm import LinearSVMModel
from pysurvival.models.simulations import SimulationModel
from pysurvival.utils.metrics import concordance_index
from sklearn.model_selection import train_test_split
from scipy.stats.stats import pearsonr

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/TGAN-imbal-AR12/TGAN-syn-AR-one-of-all-10000.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [ ]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
data_train = new_train_df
data_test = new_test_df

In [ ]:
sim = SimulationModel( survival_distribution = 'exponential',
                       risk_type = 'linear',
                       censored_parameter = 1,
                       alpha = 3)

In [ ]:
features = [f for f in data_train.columns if f not in ['a_year_time', 'a_year_event']]

time_column = 'a_year_time'
event_column = 'a_year_event'

In [ ]:
X_train, X_test = data_train[features], data_test[features]
T_train, T_test = data_train['a_year_time'].values, data_test['a_year_time'].values
E_train, E_test = data_train['a_year_event'].values, data_test['a_year_event'].values

In [ ]:
from sklearn.preprocessing import StandardScaler

X_scaler = StandardScaler().fit(X_train)
X_train = X_scaler.transform(X_train)
X_test = X_scaler.transform(X_test)

In [ ]:
svm_model = LinearSVMModel()
svm_model.fit(X_train, T_train, E_train, init_method='he_uniform',
    with_bias = True, lr = 0.5,  tol = 1e-3,  l2_reg = 1e-3)

Performing Newton-Raphson optimization: 
 * Iteration #1 - Loss = 1037443.631 - ||grad||^2 = 1894568.07784 - ||diff_w|| = 16.48260
 * Iteration #2 - Loss = 262585.193 - ||grad||^2 = 959637.03330 - ||diff_w|| = 7.40218
 * Iteration #3 - Loss = 81146.646 - ||grad||^2 = 537044.80585 - ||diff_w|| = 4.03514
 * Iteration #4 - Loss = 26004.088 - ||grad||^2 = 297232.11536 - ||diff_w|| = 2.11561
 * Iteration #5 - Loss = 10048.978 - ||grad||^2 = 145638.78966 - ||diff_w|| = 0.96455
 * Iteration #6 - Loss = 6717.297 - ||grad||^2 = 64831.75050 - ||diff_w|| = 0.44760
 * Iteration #7 - Loss = 6391.183 - ||grad||^2 = 24195.43702 - ||diff_w|| = 0.20642
 * Iteration #8 - Loss = 6432.148 - ||grad||^2 = 7903.89709 - ||diff_w|| = 0.08304
 * Iteration #9 - Loss = 6440.281 - ||grad||^2 = 2632.38014 - ||diff_w|| = 0.03669
 * Iteration #10 - Loss = 6441.143 - ||grad||^2 = 966.69576 - ||diff_w|| = 0.01805
 * Iteration #11 - Loss = 6441.345 - ||grad||^2 = 385.67540 - ||diff_w|| = 0.00930
 * Iteration #12 - Loss 

LinearSVMModel

In [ ]:
c_index = concordance_index(svm_model, X_test, T_test, E_test) #0.93
print('C-index: {:f}'.format(c_index))

C-index: 0.811919


In [ ]:
from pysurvival.utils import save_model

In [ ]:
save_model(svm_model,'/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/SSVM/SSVM_new(onehot)_tgan-imbal-10000.zip')

Saving the model to disk as /content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/SSVM/SSVM_new(onehot)_tgan-imbal-10000.zip


/usr/local/lib/python3.7/dist-packages/pysurvival/utils/__init__.py:132: FutureWarning: 'pyarrow.serialize' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  model.save(path_file)


# tgan-imbal-15000

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from pysurvival.models.svm import LinearSVMModel
from pysurvival.models.simulations import SimulationModel
from pysurvival.utils.metrics import concordance_index
from sklearn.model_selection import train_test_split
from scipy.stats.stats import pearsonr

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/TGAN-imbal-AR12/TGAN-syn-AR-one-of-all-15000.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [ ]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
data_train = new_train_df
data_test = new_test_df

In [ ]:
sim = SimulationModel( survival_distribution = 'exponential',
                       risk_type = 'linear',
                       censored_parameter = 1,
                       alpha = 3)

In [ ]:
features = [f for f in data_train.columns if f not in ['a_year_time', 'a_year_event']]

time_column = 'a_year_time'
event_column = 'a_year_event'

In [ ]:
X_train, X_test = data_train[features], data_test[features]
T_train, T_test = data_train['a_year_time'].values, data_test['a_year_time'].values
E_train, E_test = data_train['a_year_event'].values, data_test['a_year_event'].values

In [ ]:
from sklearn.preprocessing import StandardScaler

X_scaler = StandardScaler().fit(X_train)
X_train = X_scaler.transform(X_train)
X_test = X_scaler.transform(X_test)

In [ ]:
svm_model = LinearSVMModel()
svm_model.fit(X_train, T_train, E_train, init_method='he_uniform',
    with_bias = True, lr = 0.5,  tol = 1e-3,  l2_reg = 1e-3)

Performing Newton-Raphson optimization: 
 * Iteration #1 - Loss = 1112898.504 - ||grad||^2 = 1986264.42485 - ||diff_w|| = 16.46498
 * Iteration #2 - Loss = 302121.709 - ||grad||^2 = 1104407.62039 - ||diff_w|| = 6.63047
 * Iteration #3 - Loss = 94067.029 - ||grad||^2 = 614327.88572 - ||diff_w|| = 3.49475
 * Iteration #4 - Loss = 31809.090 - ||grad||^2 = 327158.62201 - ||diff_w|| = 1.64843
 * Iteration #5 - Loss = 15795.949 - ||grad||^2 = 159935.18984 - ||diff_w|| = 0.70927
 * Iteration #6 - Loss = 13298.749 - ||grad||^2 = 60992.35396 - ||diff_w|| = 0.31108
 * Iteration #7 - Loss = 13191.179 - ||grad||^2 = 19175.52951 - ||diff_w|| = 0.11864
 * Iteration #8 - Loss = 13150.443 - ||grad||^2 = 6576.44014 - ||diff_w|| = 0.04399
 * Iteration #9 - Loss = 13156.766 - ||grad||^2 = 1969.51740 - ||diff_w|| = 0.01436
 * Iteration #10 - Loss = 13157.906 - ||grad||^2 = 567.14488 - ||diff_w|| = 0.00491
 * Iteration #11 - Loss = 13158.285 - ||grad||^2 = 187.02534 - ||diff_w|| = 0.00215
 * Iteration #12 

LinearSVMModel

In [ ]:
c_index = concordance_index(svm_model, X_test, T_test, E_test) #0.93
print('C-index: {:f}'.format(c_index))

C-index: 0.807760


In [ ]:
from pysurvival.utils import save_model

In [ ]:
save_model(svm_model,'/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/SSVM/SSVM_new(onehot)_tgan-imbal-15000.zip')

Saving the model to disk as /content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/SSVM/SSVM_new(onehot)_tgan-imbal-15000.zip


/usr/local/lib/python3.7/dist-packages/pysurvival/utils/__init__.py:132: FutureWarning: 'pyarrow.serialize' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  model.save(path_file)


# tgan-imbal-20000

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from pysurvival.models.svm import LinearSVMModel
from pysurvival.models.simulations import SimulationModel
from pysurvival.utils.metrics import concordance_index
from sklearn.model_selection import train_test_split
from scipy.stats.stats import pearsonr

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/TGAN-imbal-AR12/TGAN-syn-AR-one-of-all-20000.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [ ]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
data_train = new_train_df
data_test = new_test_df

In [ ]:
sim = SimulationModel( survival_distribution = 'exponential',
                       risk_type = 'linear',
                       censored_parameter = 1,
                       alpha = 3)

In [ ]:
features = [f for f in data_train.columns if f not in ['a_year_time', 'a_year_event']]

time_column = 'a_year_time'
event_column = 'a_year_event'

In [ ]:
X_train, X_test = data_train[features], data_test[features]
T_train, T_test = data_train['a_year_time'].values, data_test['a_year_time'].values
E_train, E_test = data_train['a_year_event'].values, data_test['a_year_event'].values

In [ ]:
from sklearn.preprocessing import StandardScaler

X_scaler = StandardScaler().fit(X_train)
X_train = X_scaler.transform(X_train)
X_test = X_scaler.transform(X_test)

In [ ]:
svm_model = LinearSVMModel()
svm_model.fit(X_train, T_train, E_train, init_method='he_uniform',
    with_bias = True, lr = 0.5,  tol = 1e-3,  l2_reg = 1e-3)

Performing Newton-Raphson optimization: 
 * Iteration #1 - Loss = 2437019.685 - ||grad||^2 = 6138808.77249 - ||diff_w|| = 16.62814
 * Iteration #2 - Loss = 621739.470 - ||grad||^2 = 3188916.80616 - ||diff_w|| = 6.12318
 * Iteration #3 - Loss = 191859.842 - ||grad||^2 = 1773206.69386 - ||diff_w|| = 3.30465
 * Iteration #4 - Loss = 62761.770 - ||grad||^2 = 958056.88389 - ||diff_w|| = 1.64958
 * Iteration #5 - Loss = 28694.384 - ||grad||^2 = 492046.27346 - ||diff_w|| = 0.75561
 * Iteration #6 - Loss = 23534.985 - ||grad||^2 = 174087.14237 - ||diff_w|| = 0.37410
 * Iteration #7 - Loss = 23846.049 - ||grad||^2 = 36653.96390 - ||diff_w|| = 0.16886
 * Iteration #8 - Loss = 23825.316 - ||grad||^2 = 10227.87330 - ||diff_w|| = 0.07350
 * Iteration #9 - Loss = 23832.593 - ||grad||^2 = 3471.13124 - ||diff_w|| = 0.03565
 * Iteration #10 - Loss = 23833.037 - ||grad||^2 = 1343.32589 - ||diff_w|| = 0.01803
 * Iteration #11 - Loss = 23833.689 - ||grad||^2 = 576.16123 - ||diff_w|| = 0.00943
 * Iteration

LinearSVMModel

In [ ]:
c_index = concordance_index(svm_model, X_test, T_test, E_test) #0.93
print('C-index: {:f}'.format(c_index))

C-index: 0.808720


In [ ]:
from pysurvival.utils import save_model

In [ ]:
save_model(svm_model,'/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/SSVM/SSVM_new(onehot)_tgan-imbal-20000.zip')

Saving the model to disk as /content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/SSVM/SSVM_new(onehot)_tgan-imbal-20000.zip


/usr/local/lib/python3.7/dist-packages/pysurvival/utils/__init__.py:132: FutureWarning: 'pyarrow.serialize' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  model.save(path_file)


# tgan-imbal-25000

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from pysurvival.models.svm import LinearSVMModel
from pysurvival.models.simulations import SimulationModel
from pysurvival.utils.metrics import concordance_index
from sklearn.model_selection import train_test_split
from scipy.stats.stats import pearsonr

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/TGAN-imbal-AR12/TGAN-syn-AR-one-of-all-25000.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [ ]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
data_train = new_train_df
data_test = new_test_df

In [ ]:
sim = SimulationModel( survival_distribution = 'exponential',
                       risk_type = 'linear',
                       censored_parameter = 1,
                       alpha = 3)

In [ ]:
features = [f for f in data_train.columns if f not in ['a_year_time', 'a_year_event']]

time_column = 'a_year_time'
event_column = 'a_year_event'

In [ ]:
X_train, X_test = data_train[features], data_test[features]
T_train, T_test = data_train['a_year_time'].values, data_test['a_year_time'].values
E_train, E_test = data_train['a_year_event'].values, data_test['a_year_event'].values

In [ ]:
from sklearn.preprocessing import StandardScaler

X_scaler = StandardScaler().fit(X_train)
X_train = X_scaler.transform(X_train)
X_test = X_scaler.transform(X_test)

In [ ]:
svm_model = LinearSVMModel()
svm_model.fit(X_train, T_train, E_train, init_method='he_uniform',
    with_bias = True, lr = 0.5,  tol = 1e-3,  l2_reg = 1e-3)

Performing Newton-Raphson optimization: 
 * Iteration #1 - Loss = 2814127.212 - ||grad||^2 = 4833649.93834 - ||diff_w|| = 15.82977
 * Iteration #2 - Loss = 723067.217 - ||grad||^2 = 2472309.47157 - ||diff_w|| = 7.08957
 * Iteration #3 - Loss = 230927.102 - ||grad||^2 = 1398193.60892 - ||diff_w|| = 3.80684
 * Iteration #4 - Loss = 81631.011 - ||grad||^2 = 748029.15248 - ||diff_w|| = 1.80554
 * Iteration #5 - Loss = 42054.725 - ||grad||^2 = 308560.72222 - ||diff_w|| = 0.66845
 * Iteration #6 - Loss = 36640.233 - ||grad||^2 = 110283.68664 - ||diff_w|| = 0.26914
 * Iteration #7 - Loss = 36548.139 - ||grad||^2 = 33169.36429 - ||diff_w|| = 0.11162
 * Iteration #8 - Loss = 36571.483 - ||grad||^2 = 10416.84657 - ||diff_w|| = 0.04535
 * Iteration #9 - Loss = 36573.801 - ||grad||^2 = 3338.40852 - ||diff_w|| = 0.01966
 * Iteration #10 - Loss = 36575.674 - ||grad||^2 = 1134.61448 - ||diff_w|| = 0.00938
 * Iteration #11 - Loss = 36576.094 - ||grad||^2 = 438.47282 - ||diff_w|| = 0.00474
 * Iteration

LinearSVMModel

In [ ]:
c_index = concordance_index(svm_model, X_test, T_test, E_test) #0.93
print('C-index: {:f}'.format(c_index))

C-index: 0.810524


In [ ]:
from pysurvival.utils import save_model

In [ ]:
save_model(svm_model,'/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/SSVM/SSVM_new(onehot)_tgan-imbal-25000.zip')

Saving the model to disk as /content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/SSVM/SSVM_new(onehot)_tgan-imbal-25000.zip


/usr/local/lib/python3.7/dist-packages/pysurvival/utils/__init__.py:132: FutureWarning: 'pyarrow.serialize' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  model.save(path_file)


# tgan-imbal-30000

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from pysurvival.models.svm import LinearSVMModel
from pysurvival.models.simulations import SimulationModel
from pysurvival.utils.metrics import concordance_index
from sklearn.model_selection import train_test_split
from scipy.stats.stats import pearsonr

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/TGAN-imbal-AR12/TGAN-syn-AR-one-of-all-30000.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [ ]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
data_train = new_train_df
data_test = new_test_df

In [ ]:
sim = SimulationModel( survival_distribution = 'exponential',
                       risk_type = 'linear',
                       censored_parameter = 1,
                       alpha = 3)

In [ ]:
features = [f for f in data_train.columns if f not in ['a_year_time', 'a_year_event']]

time_column = 'a_year_time'
event_column = 'a_year_event'

In [ ]:
X_train, X_test = data_train[features], data_test[features]
T_train, T_test = data_train['a_year_time'].values, data_test['a_year_time'].values
E_train, E_test = data_train['a_year_event'].values, data_test['a_year_event'].values

In [ ]:
from sklearn.preprocessing import StandardScaler

X_scaler = StandardScaler().fit(X_train)
X_train = X_scaler.transform(X_train)
X_test = X_scaler.transform(X_test)

In [ ]:
svm_model = LinearSVMModel()
svm_model.fit(X_train, T_train, E_train, init_method='he_uniform',
    with_bias = True, lr = 0.5,  tol = 1e-3,  l2_reg = 1e-3)

Performing Newton-Raphson optimization: 
 * Iteration #1 - Loss = 5401176.483 - ||grad||^2 = 5517323.55263 - ||diff_w|| = 18.03908
 * Iteration #2 - Loss = 1381067.262 - ||grad||^2 = 2802365.13374 - ||diff_w|| = 7.44244
 * Iteration #3 - Loss = 436235.599 - ||grad||^2 = 1580232.20270 - ||diff_w|| = 4.04145
 * Iteration #4 - Loss = 149110.523 - ||grad||^2 = 888344.04550 - ||diff_w|| = 2.01322
 * Iteration #5 - Loss = 68014.754 - ||grad||^2 = 414177.57421 - ||diff_w|| = 0.74797
 * Iteration #6 - Loss = 52492.953 - ||grad||^2 = 187457.42968 - ||diff_w|| = 0.27492
 * Iteration #7 - Loss = 52121.647 - ||grad||^2 = 61358.20176 - ||diff_w|| = 0.10240
 * Iteration #8 - Loss = 52261.745 - ||grad||^2 = 16738.28469 - ||diff_w|| = 0.03142
 * Iteration #9 - Loss = 52288.762 - ||grad||^2 = 4468.21032 - ||diff_w|| = 0.01041
 * Iteration #10 - Loss = 52294.144 - ||grad||^2 = 1388.85996 - ||diff_w|| = 0.00424
 * Iteration #11 - Loss = 52295.955 - ||grad||^2 = 466.06688 - ||diff_w|| = 0.00197
 * Iterati

LinearSVMModel

In [ ]:
c_index = concordance_index(svm_model, X_test, T_test, E_test) #0.93
print('C-index: {:f}'.format(c_index))

C-index: 0.811341


In [ ]:
from pysurvival.utils import save_model

In [ ]:
save_model(svm_model,'/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/SSVM/SSVM_new(onehot)_tgan-imbal-30000.zip')

Saving the model to disk as /content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/SSVM/SSVM_new(onehot)_tgan-imbal-30000.zip


/usr/local/lib/python3.7/dist-packages/pysurvival/utils/__init__.py:132: FutureWarning: 'pyarrow.serialize' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  model.save(path_file)


# tgan-imbal-35000

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from pysurvival.models.svm import LinearSVMModel
from pysurvival.models.simulations import SimulationModel
from pysurvival.utils.metrics import concordance_index
from sklearn.model_selection import train_test_split
from scipy.stats.stats import pearsonr

In [ ]:
train_df = pd.read_csv('//content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/TGAN-imbal-AR12/TGAN-syn-AR-one-of-all-35000.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [ ]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
data_train = new_train_df
data_test = new_test_df

In [ ]:
sim = SimulationModel( survival_distribution = 'exponential',
                       risk_type = 'linear',
                       censored_parameter = 1,
                       alpha = 3)

In [ ]:
features = [f for f in data_train.columns if f not in ['a_year_time', 'a_year_event']]

time_column = 'a_year_time'
event_column = 'a_year_event'

In [ ]:
X_train, X_test = data_train[features], data_test[features]
T_train, T_test = data_train['a_year_time'].values, data_test['a_year_time'].values
E_train, E_test = data_train['a_year_event'].values, data_test['a_year_event'].values

In [ ]:
from sklearn.preprocessing import StandardScaler

X_scaler = StandardScaler().fit(X_train)
X_train = X_scaler.transform(X_train)
X_test = X_scaler.transform(X_test)

In [ ]:
svm_model = LinearSVMModel()
svm_model.fit(X_train, T_train, E_train, init_method='he_uniform',
    with_bias = True, lr = 0.5,  tol = 1e-3,  l2_reg = 1e-3)

Performing Newton-Raphson optimization: 
 * Iteration #1 - Loss = 6641973.397 - ||grad||^2 = 11563425.95038 - ||diff_w|| = 15.96463
 * Iteration #2 - Loss = 1687189.941 - ||grad||^2 = 5842693.79892 - ||diff_w|| = 7.16063
 * Iteration #3 - Loss = 526395.626 - ||grad||^2 = 3231211.73932 - ||diff_w|| = 3.87266
 * Iteration #4 - Loss = 178472.453 - ||grad||^2 = 1702821.93790 - ||diff_w|| = 1.93569
 * Iteration #5 - Loss = 88557.492 - ||grad||^2 = 842373.44488 - ||diff_w|| = 0.88290
 * Iteration #6 - Loss = 73086.757 - ||grad||^2 = 339420.44849 - ||diff_w|| = 0.42281
 * Iteration #7 - Loss = 72374.769 - ||grad||^2 = 90600.09982 - ||diff_w|| = 0.17014
 * Iteration #8 - Loss = 72594.476 - ||grad||^2 = 24108.25915 - ||diff_w|| = 0.06809
 * Iteration #9 - Loss = 72622.179 - ||grad||^2 = 7533.21429 - ||diff_w|| = 0.03036
 * Iteration #10 - Loss = 72630.223 - ||grad||^2 = 2585.54143 - ||diff_w|| = 0.01452
 * Iteration #11 - Loss = 72632.636 - ||grad||^2 = 909.12165 - ||diff_w|| = 0.00731
 * Itera

LinearSVMModel

In [ ]:
c_index = concordance_index(svm_model, X_test, T_test, E_test) #0.93
print('C-index: {:f}'.format(c_index))

C-index: 0.809235


In [ ]:
from pysurvival.utils import save_model

In [ ]:
save_model(svm_model,'/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/SSVM/SSVM_new(onehot)_tgan-imbal-35000.zip')

Saving the model to disk as /content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/SSVM/SSVM_new(onehot)_tgan-imbal-35000.zip


/usr/local/lib/python3.7/dist-packages/pysurvival/utils/__init__.py:132: FutureWarning: 'pyarrow.serialize' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  model.save(path_file)


# tgan-imbal-40000

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from pysurvival.models.svm import LinearSVMModel
from pysurvival.models.simulations import SimulationModel
from pysurvival.utils.metrics import concordance_index
from sklearn.model_selection import train_test_split
from scipy.stats.stats import pearsonr

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/TGAN-imbal-AR12/TGAN-syn-AR-one-of-all-40000.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [ ]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
data_train = new_train_df
data_test = new_test_df

In [ ]:
sim = SimulationModel( survival_distribution = 'exponential',
                       risk_type = 'linear',
                       censored_parameter = 1,
                       alpha = 3)

In [ ]:
features = [f for f in data_train.columns if f not in ['a_year_time', 'a_year_event']]

time_column = 'a_year_time'
event_column = 'a_year_event'

In [ ]:
X_train, X_test = data_train[features], data_test[features]
T_train, T_test = data_train['a_year_time'].values, data_test['a_year_time'].values
E_train, E_test = data_train['a_year_event'].values, data_test['a_year_event'].values

In [ ]:
from sklearn.preprocessing import StandardScaler

X_scaler = StandardScaler().fit(X_train)
X_train = X_scaler.transform(X_train)
X_test = X_scaler.transform(X_test)

In [ ]:
svm_model = LinearSVMModel()
svm_model.fit(X_train, T_train, E_train, init_method='he_uniform',
    with_bias = True, lr = 0.5,  tol = 1e-3,  l2_reg = 1e-3)

Performing Newton-Raphson optimization: 
 * Iteration #1 - Loss = 10308385.192 - ||grad||^2 = 17074543.92509 - ||diff_w|| = 16.43558
 * Iteration #2 - Loss = 2873111.744 - ||grad||^2 = 9614405.28066 - ||diff_w|| = 6.82797
 * Iteration #3 - Loss = 881934.302 - ||grad||^2 = 5357465.25841 - ||diff_w|| = 3.68179
 * Iteration #4 - Loss = 283335.045 - ||grad||^2 = 2889476.59256 - ||diff_w|| = 1.81828
 * Iteration #5 - Loss = 125945.200 - ||grad||^2 = 1456919.50874 - ||diff_w|| = 0.81098
 * Iteration #6 - Loss = 96993.183 - ||grad||^2 = 629854.15032 - ||diff_w|| = 0.40479
 * Iteration #7 - Loss = 94226.800 - ||grad||^2 = 240699.88717 - ||diff_w|| = 0.17678
 * Iteration #8 - Loss = 94191.273 - ||grad||^2 = 83877.74519 - ||diff_w|| = 0.06726
 * Iteration #9 - Loss = 94242.709 - ||grad||^2 = 25148.70106 - ||diff_w|| = 0.02437
 * Iteration #10 - Loss = 94276.307 - ||grad||^2 = 7485.98525 - ||diff_w|| = 0.00889
 * Iteration #11 - Loss = 94286.761 - ||grad||^2 = 2720.73645 - ||diff_w|| = 0.00387
 *

LinearSVMModel

In [ ]:
c_index = concordance_index(svm_model, X_test, T_test, E_test) #0.93
print('C-index: {:f}'.format(c_index))

C-index: 0.809564


In [ ]:
from pysurvival.utils import save_model

In [ ]:
save_model(svm_model,'/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/SSVM/SSVM_new(onehot)_tgan-imbal-40000.zip')

Saving the model to disk as /content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/SSVM/SSVM_new(onehot)_tgan-imbal-40000.zip


/usr/local/lib/python3.7/dist-packages/pysurvival/utils/__init__.py:132: FutureWarning: 'pyarrow.serialize' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  model.save(path_file)


# tgan-imbal-45000

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from pysurvival.models.svm import LinearSVMModel
from pysurvival.models.simulations import SimulationModel
from pysurvival.utils.metrics import concordance_index
from sklearn.model_selection import train_test_split
from scipy.stats.stats import pearsonr

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/TGAN-imbal-AR12/TGAN-syn-AR-one-of-all-45000.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [ ]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
data_train = new_train_df
data_test = new_test_df

In [ ]:
sim = SimulationModel( survival_distribution = 'exponential',
                       risk_type = 'linear',
                       censored_parameter = 1,
                       alpha = 3)

In [ ]:
features = [f for f in data_train.columns if f not in ['a_year_time', 'a_year_event']]

time_column = 'a_year_time'
event_column = 'a_year_event'

In [ ]:
X_train, X_test = data_train[features], data_test[features]
T_train, T_test = data_train['a_year_time'].values, data_test['a_year_time'].values
E_train, E_test = data_train['a_year_event'].values, data_test['a_year_event'].values

In [ ]:
from sklearn.preprocessing import StandardScaler

X_scaler = StandardScaler().fit(X_train)
X_train = X_scaler.transform(X_train)
X_test = X_scaler.transform(X_test)

In [ ]:
svm_model = LinearSVMModel()
svm_model.fit(X_train, T_train, E_train, init_method='he_uniform',
    with_bias = True, lr = 0.5,  tol = 1e-3,  l2_reg = 1e-3)

Performing Newton-Raphson optimization: 
 * Iteration #1 - Loss = 47084893.526 - ||grad||^2 = 75031729.52377 - ||diff_w|| = 16.19928
 * Iteration #2 - Loss = 11871731.519 - ||grad||^2 = 37951189.63233 - ||diff_w|| = 7.27697
 * Iteration #3 - Loss = 3632200.108 - ||grad||^2 = 21218610.33017 - ||diff_w|| = 3.97565
 * Iteration #4 - Loss = 1119828.480 - ||grad||^2 = 11858357.77672 - ||diff_w|| = 2.12916
 * Iteration #5 - Loss = 368282.680 - ||grad||^2 = 6390286.99202 - ||diff_w|| = 1.07676
 * Iteration #6 - Loss = 164246.698 - ||grad||^2 = 3270090.28114 - ||diff_w|| = 0.52811
 * Iteration #7 - Loss = 121858.601 - ||grad||^2 = 1479715.59509 - ||diff_w|| = 0.27064
 * Iteration #8 - Loss = 120560.720 - ||grad||^2 = 471823.83652 - ||diff_w|| = 0.11612
 * Iteration #9 - Loss = 121358.949 - ||grad||^2 = 130170.02764 - ||diff_w|| = 0.04584
 * Iteration #10 - Loss = 121353.921 - ||grad||^2 = 35057.78439 - ||diff_w|| = 0.01702
 * Iteration #11 - Loss = 121387.922 - ||grad||^2 = 8265.86683 - ||diff

LinearSVMModel

In [ ]:
c_index = concordance_index(svm_model, X_test, T_test, E_test) #0.93
print('C-index: {:f}'.format(c_index))

C-index: 0.810222


In [ ]:
from pysurvival.utils import save_model

In [ ]:
save_model(svm_model,'/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/SSVM/SSVM_new(onehot)_tgan-imbal-45000.zip')

Saving the model to disk as /content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/SSVM/SSVM_new(onehot)_tgan-imbal-45000.zip


/usr/local/lib/python3.7/dist-packages/pysurvival/utils/__init__.py:132: FutureWarning: 'pyarrow.serialize' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  model.save(path_file)


# tgan-imbal-50000

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from pysurvival.models.svm import LinearSVMModel
from pysurvival.models.simulations import SimulationModel
from pysurvival.utils.metrics import concordance_index
from sklearn.model_selection import train_test_split
from scipy.stats.stats import pearsonr

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/TGAN-imbal-AR12/TGAN-syn-AR-one-of-all-50000.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [ ]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
data_train = new_train_df
data_test = new_test_df

In [ ]:
sim = SimulationModel( survival_distribution = 'exponential',
                       risk_type = 'linear',
                       censored_parameter = 1,
                       alpha = 3)

In [ ]:
features = [f for f in data_train.columns if f not in ['a_year_time', 'a_year_event']]

time_column = 'a_year_time'
event_column = 'a_year_event'

In [ ]:
X_train, X_test = data_train[features], data_test[features]
T_train, T_test = data_train['a_year_time'].values, data_test['a_year_time'].values
E_train, E_test = data_train['a_year_event'].values, data_test['a_year_event'].values

In [ ]:
from sklearn.preprocessing import StandardScaler

X_scaler = StandardScaler().fit(X_train)
X_train = X_scaler.transform(X_train)
X_test = X_scaler.transform(X_test)

In [ ]:
svm_model = LinearSVMModel()
svm_model.fit(X_train, T_train, E_train, init_method='he_uniform',
    with_bias = True, lr = 0.5,  tol = 1e-3,  l2_reg = 1e-3)

Performing Newton-Raphson optimization: 
 * Iteration #1 - Loss = 29391470.428 - ||grad||^2 = 50024734.22640 - ||diff_w|| = 16.25078
 * Iteration #2 - Loss = 7404865.244 - ||grad||^2 = 25257550.99939 - ||diff_w|| = 7.30033
 * Iteration #3 - Loss = 2263005.907 - ||grad||^2 = 14050738.91335 - ||diff_w|| = 3.98300
 * Iteration #4 - Loss = 703056.727 - ||grad||^2 = 7730104.09819 - ||diff_w|| = 2.08552
 * Iteration #5 - Loss = 253584.873 - ||grad||^2 = 3992389.37087 - ||diff_w|| = 0.93287
 * Iteration #6 - Loss = 154586.046 - ||grad||^2 = 1828511.38705 - ||diff_w|| = 0.43548
 * Iteration #7 - Loss = 143985.523 - ||grad||^2 = 607842.98405 - ||diff_w|| = 0.21831
 * Iteration #8 - Loss = 144563.742 - ||grad||^2 = 175819.69929 - ||diff_w|| = 0.08961
 * Iteration #9 - Loss = 144703.272 - ||grad||^2 = 53133.13515 - ||diff_w|| = 0.03100
 * Iteration #10 - Loss = 144761.765 - ||grad||^2 = 16716.56826 - ||diff_w|| = 0.01149
 * Iteration #11 - Loss = 144769.875 - ||grad||^2 = 5482.69553 - ||diff_w|| 

LinearSVMModel

In [ ]:
c_index = concordance_index(svm_model, X_test, T_test, E_test) #0.93
print('C-index: {:f}'.format(c_index))

C-index: 0.811208


In [ ]:
from pysurvival.utils import save_model

In [ ]:
save_model(svm_model,'/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/SSVM/SSVM_new(onehot)_tgan-imbal-50000.zip')

Saving the model to disk as /content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/SSVM/SSVM_new(onehot)_tgan-imbal-50000.zip


/usr/local/lib/python3.7/dist-packages/pysurvival/utils/__init__.py:132: FutureWarning: 'pyarrow.serialize' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  model.save(path_file)


# tgan-bal-5000

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from pysurvival.models.svm import LinearSVMModel
from pysurvival.models.simulations import SimulationModel
from pysurvival.utils.metrics import concordance_index
from sklearn.model_selection import train_test_split
from scipy.stats.stats import pearsonr

In [ ]:
train_df = pd.read_csv('//content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/TGAN-bala-AR12/TGAN-syn-balanceAR-all-in-one-5000.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [ ]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
data_train = new_train_df
data_test = new_test_df

In [ ]:
sim = SimulationModel( survival_distribution = 'exponential',
                       risk_type = 'linear',
                       censored_parameter = 1,
                       alpha = 3)

In [ ]:
features = [f for f in data_train.columns if f not in ['a_year_time', 'a_year_event']]

time_column = 'a_year_time'
event_column = 'a_year_event'

In [ ]:
X_train, X_test = data_train[features], data_test[features]
T_train, T_test = data_train['a_year_time'].values, data_test['a_year_time'].values
E_train, E_test = data_train['a_year_event'].values, data_test['a_year_event'].values

In [ ]:
from sklearn.preprocessing import StandardScaler

X_scaler = StandardScaler().fit(X_train)
X_train = X_scaler.transform(X_train)
X_test = X_scaler.transform(X_test)

In [ ]:
svm_model = LinearSVMModel()
svm_model.fit(X_train, T_train, E_train, init_method='he_uniform',
    with_bias = True, lr = 0.5,  tol = 1e-3,  l2_reg = 1e-3)

Performing Newton-Raphson optimization: 
 * Iteration #1 - Loss = 224510.776 - ||grad||^2 = 244451.76548 - ||diff_w|| = 19.68979
 * Iteration #2 - Loss = 57629.245 - ||grad||^2 = 126217.82593 - ||diff_w|| = 8.82544
 * Iteration #3 - Loss = 18358.963 - ||grad||^2 = 72992.52614 - ||diff_w|| = 4.76811
 * Iteration #4 - Loss = 6373.831 - ||grad||^2 = 41910.76844 - ||diff_w|| = 2.36247
 * Iteration #5 - Loss = 3024.120 - ||grad||^2 = 18596.54884 - ||diff_w|| = 0.93165
 * Iteration #6 - Loss = 2383.743 - ||grad||^2 = 7218.71708 - ||diff_w|| = 0.36032
 * Iteration #7 - Loss = 2320.974 - ||grad||^2 = 2886.37890 - ||diff_w|| = 0.15738
 * Iteration #8 - Loss = 2314.951 - ||grad||^2 = 1099.32474 - ||diff_w|| = 0.07090
 * Iteration #9 - Loss = 2314.749 - ||grad||^2 = 405.87516 - ||diff_w|| = 0.03144
 * Iteration #10 - Loss = 2314.880 - ||grad||^2 = 148.86797 - ||diff_w|| = 0.01432
 * Iteration #11 - Loss = 2315.027 - ||grad||^2 = 53.25067 - ||diff_w|| = 0.00676
 * Iteration #12 - Loss = 2315.040 -

LinearSVMModel

In [ ]:
c_index = concordance_index(svm_model, X_test, T_test, E_test) #0.93
print('C-index: {:f}'.format(c_index))

C-index: 0.829151


In [ ]:
from pysurvival.utils import save_model

In [ ]:
save_model(svm_model,'/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/SSVM/SSVM_new(onehot)_tgan-bal-5000.zip')

Saving the model to disk as /content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/SSVM/SSVM_new(onehot)_tgan-bal-5000.zip


/usr/local/lib/python3.7/dist-packages/pysurvival/utils/__init__.py:132: FutureWarning: 'pyarrow.serialize' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  model.save(path_file)


# tgan-bal-10000

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from pysurvival.models.svm import LinearSVMModel
from pysurvival.models.simulations import SimulationModel
from pysurvival.utils.metrics import concordance_index
from sklearn.model_selection import train_test_split
from scipy.stats.stats import pearsonr

In [ ]:
train_df = pd.read_csv('//content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/TGAN-bala-AR12/TGAN-syn-balanceAR-all-in-one-10000.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [ ]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
data_train = new_train_df
data_test = new_test_df

In [ ]:
sim = SimulationModel( survival_distribution = 'exponential',
                       risk_type = 'linear',
                       censored_parameter = 1,
                       alpha = 3)

In [ ]:
features = [f for f in data_train.columns if f not in ['a_year_time', 'a_year_event']]

time_column = 'a_year_time'
event_column = 'a_year_event'

In [ ]:
X_train, X_test = data_train[features], data_test[features]
T_train, T_test = data_train['a_year_time'].values, data_test['a_year_time'].values
E_train, E_test = data_train['a_year_event'].values, data_test['a_year_event'].values

In [ ]:
from sklearn.preprocessing import StandardScaler

X_scaler = StandardScaler().fit(X_train)
X_train = X_scaler.transform(X_train)
X_test = X_scaler.transform(X_test)

In [ ]:
svm_model = LinearSVMModel()
svm_model.fit(X_train, T_train, E_train, init_method='he_uniform',
    with_bias = True, lr = 0.5,  tol = 1e-3,  l2_reg = 1e-3)

Performing Newton-Raphson optimization: 
 * Iteration #1 - Loss = 607933.691 - ||grad||^2 = 885780.07928 - ||diff_w|| = 14.24531
 * Iteration #2 - Loss = 159232.596 - ||grad||^2 = 462922.68343 - ||diff_w|| = 6.36933
 * Iteration #3 - Loss = 52791.159 - ||grad||^2 = 272734.20389 - ||diff_w|| = 3.38912
 * Iteration #4 - Loss = 19963.826 - ||grad||^2 = 157614.96015 - ||diff_w|| = 1.54245
 * Iteration #5 - Loss = 10728.212 - ||grad||^2 = 63650.93927 - ||diff_w|| = 0.46589
 * Iteration #6 - Loss = 9161.648 - ||grad||^2 = 19891.62542 - ||diff_w|| = 0.17196
 * Iteration #7 - Loss = 9086.566 - ||grad||^2 = 8039.12211 - ||diff_w|| = 0.07742
 * Iteration #8 - Loss = 9081.070 - ||grad||^2 = 3034.86461 - ||diff_w|| = 0.03376
 * Iteration #9 - Loss = 9080.209 - ||grad||^2 = 1189.43457 - ||diff_w|| = 0.01421
 * Iteration #10 - Loss = 9079.982 - ||grad||^2 = 533.12503 - ||diff_w|| = 0.00673
 * Iteration #11 - Loss = 9079.725 - ||grad||^2 = 249.76003 - ||diff_w|| = 0.00334
 * Iteration #12 - Loss = 90

LinearSVMModel

In [ ]:
c_index = concordance_index(svm_model, X_test, T_test, E_test) #0.93
print('C-index: {:f}'.format(c_index))

C-index: 0.828831


In [ ]:
from pysurvival.utils import save_model

In [ ]:
save_model(svm_model,'/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/SSVM/SSVM_new(onehot)_tgan-bal-10000.zip')

Saving the model to disk as /content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/SSVM/SSVM_new(onehot)_tgan-bal-10000.zip


/usr/local/lib/python3.7/dist-packages/pysurvival/utils/__init__.py:132: FutureWarning: 'pyarrow.serialize' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  model.save(path_file)


# tgan-bal-15000

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from pysurvival.models.svm import LinearSVMModel
from pysurvival.models.simulations import SimulationModel
from pysurvival.utils.metrics import concordance_index
from sklearn.model_selection import train_test_split
from scipy.stats.stats import pearsonr

In [ ]:
train_df = pd.read_csv('//content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/TGAN-bala-AR12/TGAN-syn-balanceAR-all-in-one-15000.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [ ]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
data_train = new_train_df
data_test = new_test_df

In [ ]:
sim = SimulationModel( survival_distribution = 'exponential',
                       risk_type = 'linear',
                       censored_parameter = 1,
                       alpha = 3)

In [ ]:
features = [f for f in data_train.columns if f not in ['a_year_time', 'a_year_event']]

time_column = 'a_year_time'
event_column = 'a_year_event'

In [ ]:
X_train, X_test = data_train[features], data_test[features]
T_train, T_test = data_train['a_year_time'].values, data_test['a_year_time'].values
E_train, E_test = data_train['a_year_event'].values, data_test['a_year_event'].values

In [ ]:
from sklearn.preprocessing import StandardScaler

X_scaler = StandardScaler().fit(X_train)
X_train = X_scaler.transform(X_train)
X_test = X_scaler.transform(X_test)

In [ ]:
svm_model = LinearSVMModel()
svm_model.fit(X_train, T_train, E_train, init_method='he_uniform',
    with_bias = True, lr = 0.5,  tol = 1e-3,  l2_reg = 1e-3)

Performing Newton-Raphson optimization: 
 * Iteration #1 - Loss = 2730292.212 - ||grad||^2 = 4449011.00198 - ||diff_w|| = 15.31328
 * Iteration #2 - Loss = 691668.998 - ||grad||^2 = 2254843.31014 - ||diff_w|| = 6.86710
 * Iteration #3 - Loss = 213340.985 - ||grad||^2 = 1258438.70540 - ||diff_w|| = 3.71780
 * Iteration #4 - Loss = 68175.352 - ||grad||^2 = 682708.07315 - ||diff_w|| = 1.87694
 * Iteration #5 - Loss = 28573.760 - ||grad||^2 = 332114.83956 - ||diff_w|| = 0.77577
 * Iteration #6 - Loss = 21462.682 - ||grad||^2 = 140214.79859 - ||diff_w|| = 0.36143
 * Iteration #7 - Loss = 20652.048 - ||grad||^2 = 48358.12991 - ||diff_w|| = 0.16251
 * Iteration #8 - Loss = 20552.344 - ||grad||^2 = 16548.06794 - ||diff_w|| = 0.06844
 * Iteration #9 - Loss = 20540.234 - ||grad||^2 = 5883.01360 - ||diff_w|| = 0.02652
 * Iteration #10 - Loss = 20539.398 - ||grad||^2 = 2286.50540 - ||diff_w|| = 0.01035
 * Iteration #11 - Loss = 20537.731 - ||grad||^2 = 939.58506 - ||diff_w|| = 0.00430
 * Iteration

LinearSVMModel

In [ ]:
c_index = concordance_index(svm_model, X_test, T_test, E_test) #0.93
print('C-index: {:f}'.format(c_index))

C-index: 0.831292


In [ ]:
from pysurvival.utils import save_model

In [ ]:
save_model(svm_model,'/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/SSVM/SSVM_new(onehot)_tgan-bal-15000.zip')

Saving the model to disk as /content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/SSVM/SSVM_new(onehot)_tgan-bal-15000.zip


/usr/local/lib/python3.7/dist-packages/pysurvival/utils/__init__.py:132: FutureWarning: 'pyarrow.serialize' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  model.save(path_file)


# tgan-bal-20000

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from pysurvival.models.svm import LinearSVMModel
from pysurvival.models.simulations import SimulationModel
from pysurvival.utils.metrics import concordance_index
from sklearn.model_selection import train_test_split
from scipy.stats.stats import pearsonr

In [ ]:
train_df = pd.read_csv('//content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/TGAN-bala-AR12/TGAN-syn-balanceAR-all-in-one-20000.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [ ]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
data_train = new_train_df
data_test = new_test_df

In [ ]:
sim = SimulationModel( survival_distribution = 'exponential',
                       risk_type = 'linear',
                       censored_parameter = 1,
                       alpha = 3)

In [ ]:
features = [f for f in data_train.columns if f not in ['a_year_time', 'a_year_event']]

time_column = 'a_year_time'
event_column = 'a_year_event'

In [ ]:
X_train, X_test = data_train[features], data_test[features]
T_train, T_test = data_train['a_year_time'].values, data_test['a_year_time'].values
E_train, E_test = data_train['a_year_event'].values, data_test['a_year_event'].values

In [ ]:
from sklearn.preprocessing import StandardScaler

X_scaler = StandardScaler().fit(X_train)
X_train = X_scaler.transform(X_train)
X_test = X_scaler.transform(X_test)

In [ ]:
svm_model = LinearSVMModel()
svm_model.fit(X_train, T_train, E_train, init_method='he_uniform',
    with_bias = True, lr = 0.5,  tol = 1e-3,  l2_reg = 1e-3)

Performing Newton-Raphson optimization: 
 * Iteration #1 - Loss = 3048455.282 - ||grad||^2 = 6224360.02327 - ||diff_w|| = 14.32122
 * Iteration #2 - Loss = 774595.981 - ||grad||^2 = 3162981.75133 - ||diff_w|| = 6.39161
 * Iteration #3 - Loss = 242397.825 - ||grad||^2 = 1763339.95828 - ||diff_w|| = 3.37600
 * Iteration #4 - Loss = 83864.950 - ||grad||^2 = 945843.67720 - ||diff_w|| = 1.54992
 * Iteration #5 - Loss = 43800.229 - ||grad||^2 = 476706.00065 - ||diff_w|| = 0.67930
 * Iteration #6 - Loss = 37219.386 - ||grad||^2 = 188254.31099 - ||diff_w|| = 0.33241
 * Iteration #7 - Loss = 36561.844 - ||grad||^2 = 66424.08732 - ||diff_w|| = 0.13910
 * Iteration #8 - Loss = 36356.485 - ||grad||^2 = 23612.60097 - ||diff_w|| = 0.05567
 * Iteration #9 - Loss = 36360.952 - ||grad||^2 = 7783.21745 - ||diff_w|| = 0.02163
 * Iteration #10 - Loss = 36367.148 - ||grad||^2 = 2413.42350 - ||diff_w|| = 0.00767
 * Iteration #11 - Loss = 36368.823 - ||grad||^2 = 837.96493 - ||diff_w|| = 0.00303
 * Iteration

LinearSVMModel

In [ ]:
c_index = concordance_index(svm_model, X_test, T_test, E_test) #0.93
print('C-index: {:f}'.format(c_index))

C-index: 0.829328


In [ ]:
from pysurvival.utils import save_model

In [ ]:
save_model(svm_model,'/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/SSVM/SSVM_new(onehot)_tgan-bal-20000.zip')

Saving the model to disk as /content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/SSVM/SSVM_new(onehot)_tgan-bal-20000.zip


/usr/local/lib/python3.7/dist-packages/pysurvival/utils/__init__.py:132: FutureWarning: 'pyarrow.serialize' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  model.save(path_file)


# tgan-bal-25000

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from pysurvival.models.svm import LinearSVMModel
from pysurvival.models.simulations import SimulationModel
from pysurvival.utils.metrics import concordance_index
from sklearn.model_selection import train_test_split
from scipy.stats.stats import pearsonr

In [ ]:
train_df = pd.read_csv('//content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/TGAN-bala-AR12/TGAN-syn-balanceAR-all-in-one-25000.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [ ]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
data_train = new_train_df
data_test = new_test_df

In [ ]:
sim = SimulationModel( survival_distribution = 'exponential',
                       risk_type = 'linear',
                       censored_parameter = 1,
                       alpha = 3)

In [ ]:
features = [f for f in data_train.columns if f not in ['a_year_time', 'a_year_event']]

time_column = 'a_year_time'
event_column = 'a_year_event'

In [ ]:
X_train, X_test = data_train[features], data_test[features]
T_train, T_test = data_train['a_year_time'].values, data_test['a_year_time'].values
E_train, E_test = data_train['a_year_event'].values, data_test['a_year_event'].values

In [ ]:
from sklearn.preprocessing import StandardScaler

X_scaler = StandardScaler().fit(X_train)
X_train = X_scaler.transform(X_train)
X_test = X_scaler.transform(X_test)

In [ ]:
svm_model = LinearSVMModel()
svm_model.fit(X_train, T_train, E_train, init_method='he_uniform',
    with_bias = True, lr = 0.5,  tol = 1e-3,  l2_reg = 1e-3)

Performing Newton-Raphson optimization: 
 * Iteration #1 - Loss = 4941513.516 - ||grad||^2 = 8740792.46278 - ||diff_w|| = 16.00825
 * Iteration #2 - Loss = 1266781.383 - ||grad||^2 = 4517305.23222 - ||diff_w|| = 6.39477
 * Iteration #3 - Loss = 399018.183 - ||grad||^2 = 2551443.93539 - ||diff_w|| = 3.46474
 * Iteration #4 - Loss = 137090.188 - ||grad||^2 = 1384643.43991 - ||diff_w|| = 1.70925
 * Iteration #5 - Loss = 69538.968 - ||grad||^2 = 605142.37187 - ||diff_w|| = 0.64597
 * Iteration #6 - Loss = 59804.540 - ||grad||^2 = 259868.08840 - ||diff_w|| = 0.28746
 * Iteration #7 - Loss = 58850.402 - ||grad||^2 = 102973.92609 - ||diff_w|| = 0.13490
 * Iteration #8 - Loss = 58788.339 - ||grad||^2 = 38719.73678 - ||diff_w|| = 0.06188
 * Iteration #9 - Loss = 58760.902 - ||grad||^2 = 15151.96975 - ||diff_w|| = 0.02931
 * Iteration #10 - Loss = 58754.881 - ||grad||^2 = 6350.17720 - ||diff_w|| = 0.01478
 * Iteration #11 - Loss = 58752.515 - ||grad||^2 = 2823.76823 - ||diff_w|| = 0.00768
 * Ite

LinearSVMModel

In [ ]:
c_index = concordance_index(svm_model, X_test, T_test, E_test) #0.93
print('C-index: {:f}'.format(c_index))

C-index: 0.829097


In [ ]:
from pysurvival.utils import save_model

In [ ]:
save_model(svm_model,'/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/SSVM/SSVM_new(onehot)_tgan-bal-25000.zip')

Saving the model to disk as /content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/SSVM/SSVM_new(onehot)_tgan-bal-25000.zip


/usr/local/lib/python3.7/dist-packages/pysurvival/utils/__init__.py:132: FutureWarning: 'pyarrow.serialize' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  model.save(path_file)


# tgan-bal-30000

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from pysurvival.models.svm import LinearSVMModel
from pysurvival.models.simulations import SimulationModel
from pysurvival.utils.metrics import concordance_index
from sklearn.model_selection import train_test_split
from scipy.stats.stats import pearsonr

In [ ]:
train_df = pd.read_csv('//content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/TGAN-bala-AR12/TGAN-syn-balanceAR-all-in-one-30000.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [ ]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
data_train = new_train_df
data_test = new_test_df

In [ ]:
sim = SimulationModel( survival_distribution = 'exponential',
                       risk_type = 'linear',
                       censored_parameter = 1,
                       alpha = 3)

In [ ]:
features = [f for f in data_train.columns if f not in ['a_year_time', 'a_year_event']]

time_column = 'a_year_time'
event_column = 'a_year_event'

In [ ]:
X_train, X_test = data_train[features], data_test[features]
T_train, T_test = data_train['a_year_time'].values, data_test['a_year_time'].values
E_train, E_test = data_train['a_year_event'].values, data_test['a_year_event'].values

In [ ]:
from sklearn.preprocessing import StandardScaler

X_scaler = StandardScaler().fit(X_train)
X_train = X_scaler.transform(X_train)
X_test = X_scaler.transform(X_test)

In [ ]:
svm_model = LinearSVMModel()
svm_model.fit(X_train, T_train, E_train, init_method='he_uniform',
    with_bias = True, lr = 0.5,  tol = 1e-3,  l2_reg = 1e-3)

Performing Newton-Raphson optimization: 
 * Iteration #1 - Loss = 12850433.167 - ||grad||^2 = 22183218.61383 - ||diff_w|| = 15.25584
 * Iteration #2 - Loss = 3269009.179 - ||grad||^2 = 11297951.34272 - ||diff_w|| = 6.83850
 * Iteration #3 - Loss = 1017914.772 - ||grad||^2 = 6372091.17640 - ||diff_w|| = 3.69278
 * Iteration #4 - Loss = 327744.597 - ||grad||^2 = 3544762.41354 - ||diff_w|| = 1.90102
 * Iteration #5 - Loss = 126228.406 - ||grad||^2 = 1652505.48772 - ||diff_w|| = 0.84188
 * Iteration #6 - Loss = 86022.732 - ||grad||^2 = 752819.65775 - ||diff_w|| = 0.36415
 * Iteration #7 - Loss = 80543.423 - ||grad||^2 = 299902.28540 - ||diff_w|| = 0.15934
 * Iteration #8 - Loss = 80621.713 - ||grad||^2 = 102153.31298 - ||diff_w|| = 0.06197
 * Iteration #9 - Loss = 80681.996 - ||grad||^2 = 35662.84857 - ||diff_w|| = 0.02493
 * Iteration #10 - Loss = 80671.865 - ||grad||^2 = 13660.40833 - ||diff_w|| = 0.01177
 * Iteration #11 - Loss = 80667.113 - ||grad||^2 = 5695.92374 - ||diff_w|| = 0.0061

LinearSVMModel

In [ ]:
c_index = concordance_index(svm_model, X_test, T_test, E_test) #0.93
print('C-index: {:f}'.format(c_index))

C-index: 0.830297


In [ ]:
from pysurvival.utils import save_model

In [ ]:
save_model(svm_model,'/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/SSVM/SSVM_new(onehot)_tgan-bal-30000.zip')

Saving the model to disk as /content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/SSVM/SSVM_new(onehot)_tgan-bal-30000.zip


/usr/local/lib/python3.7/dist-packages/pysurvival/utils/__init__.py:132: FutureWarning: 'pyarrow.serialize' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  model.save(path_file)


# tgan-bal-35000

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from pysurvival.models.svm import LinearSVMModel
from pysurvival.models.simulations import SimulationModel
from pysurvival.utils.metrics import concordance_index
from sklearn.model_selection import train_test_split
from scipy.stats.stats import pearsonr

In [ ]:
train_df = pd.read_csv('//content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/TGAN-bala-AR12/TGAN-syn-balanceAR-all-in-one-35000.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [ ]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
data_train = new_train_df
data_test = new_test_df

In [ ]:
sim = SimulationModel( survival_distribution = 'exponential',
                       risk_type = 'linear',
                       censored_parameter = 1,
                       alpha = 3)

In [ ]:
features = [f for f in data_train.columns if f not in ['a_year_time', 'a_year_event']]

time_column = 'a_year_time'
event_column = 'a_year_event'

In [ ]:
X_train, X_test = data_train[features], data_test[features]
T_train, T_test = data_train['a_year_time'].values, data_test['a_year_time'].values
E_train, E_test = data_train['a_year_event'].values, data_test['a_year_event'].values

In [ ]:
from sklearn.preprocessing import StandardScaler

X_scaler = StandardScaler().fit(X_train)
X_train = X_scaler.transform(X_train)
X_test = X_scaler.transform(X_test)

In [ ]:
svm_model = LinearSVMModel()
svm_model.fit(X_train, T_train, E_train, init_method='he_uniform',
    with_bias = True, lr = 0.5,  tol = 1e-3,  l2_reg = 1e-3)

Performing Newton-Raphson optimization: 
 * Iteration #1 - Loss = 9914751.950 - ||grad||^2 = 15175693.83176 - ||diff_w|| = 19.07061
 * Iteration #2 - Loss = 2538373.982 - ||grad||^2 = 7708938.01978 - ||diff_w|| = 8.54852
 * Iteration #3 - Loss = 804381.959 - ||grad||^2 = 4312523.04383 - ||diff_w|| = 4.60904
 * Iteration #4 - Loss = 277819.511 - ||grad||^2 = 2309097.97135 - ||diff_w|| = 2.23292
 * Iteration #5 - Loss = 136825.814 - ||grad||^2 = 1078372.73627 - ||diff_w|| = 0.78046
 * Iteration #6 - Loss = 114336.663 - ||grad||^2 = 444922.49786 - ||diff_w|| = 0.28368
 * Iteration #7 - Loss = 112764.835 - ||grad||^2 = 147322.86196 - ||diff_w|| = 0.11306
 * Iteration #8 - Loss = 112482.321 - ||grad||^2 = 50840.51580 - ||diff_w|| = 0.04580
 * Iteration #9 - Loss = 112482.417 - ||grad||^2 = 16989.62173 - ||diff_w|| = 0.01801
 * Iteration #10 - Loss = 112488.074 - ||grad||^2 = 5806.24787 - ||diff_w|| = 0.00732
 * Iteration #11 - Loss = 112489.464 - ||grad||^2 = 2142.77532 - ||diff_w|| = 0.003

LinearSVMModel

In [ ]:
c_index = concordance_index(svm_model, X_test, T_test, E_test) #0.93
print('C-index: {:f}'.format(c_index))

C-index: 0.829115


In [ ]:
from pysurvival.utils import save_model

In [ ]:
save_model(svm_model,'/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/SSVM/SSVM_new(onehot)_tgan-bal-35000.zip')

Saving the model to disk as /content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/SSVM/SSVM_new(onehot)_tgan-bal-35000.zip


/usr/local/lib/python3.7/dist-packages/pysurvival/utils/__init__.py:132: FutureWarning: 'pyarrow.serialize' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  model.save(path_file)


# tgan-bal-40000

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from pysurvival.models.svm import LinearSVMModel
from pysurvival.models.simulations import SimulationModel
from pysurvival.utils.metrics import concordance_index
from sklearn.model_selection import train_test_split
from scipy.stats.stats import pearsonr

In [ ]:
train_df = pd.read_csv('//content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/TGAN-bala-AR12/TGAN-syn-balanceAR-all-in-one-40000.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [ ]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
data_train = new_train_df
data_test = new_test_df

In [ ]:
sim = SimulationModel( survival_distribution = 'exponential',
                       risk_type = 'linear',
                       censored_parameter = 1,
                       alpha = 3)

In [ ]:
features = [f for f in data_train.columns if f not in ['a_year_time', 'a_year_event']]

time_column = 'a_year_time'
event_column = 'a_year_event'

In [ ]:
X_train, X_test = data_train[features], data_test[features]
T_train, T_test = data_train['a_year_time'].values, data_test['a_year_time'].values
E_train, E_test = data_train['a_year_event'].values, data_test['a_year_event'].values

In [ ]:
from sklearn.preprocessing import StandardScaler

X_scaler = StandardScaler().fit(X_train)
X_train = X_scaler.transform(X_train)
X_test = X_scaler.transform(X_test)

In [ ]:
svm_model = LinearSVMModel()
svm_model.fit(X_train, T_train, E_train, init_method='he_uniform',
    with_bias = True, lr = 0.5,  tol = 1e-3,  l2_reg = 1e-3)

Performing Newton-Raphson optimization: 
 * Iteration #1 - Loss = 16129307.255 - ||grad||^2 = 22817339.42563 - ||diff_w|| = 18.03262
 * Iteration #2 - Loss = 4092296.446 - ||grad||^2 = 11552486.81270 - ||diff_w|| = 8.07817
 * Iteration #3 - Loss = 1276502.520 - ||grad||^2 = 6435031.35490 - ||diff_w|| = 4.34839
 * Iteration #4 - Loss = 429765.323 - ||grad||^2 = 3501192.90576 - ||diff_w|| = 2.14198
 * Iteration #5 - Loss = 199907.425 - ||grad||^2 = 1778987.06589 - ||diff_w|| = 0.90157
 * Iteration #6 - Loss = 157439.284 - ||grad||^2 = 738277.39330 - ||diff_w|| = 0.42635
 * Iteration #7 - Loss = 153371.576 - ||grad||^2 = 252594.83933 - ||diff_w|| = 0.19264
 * Iteration #8 - Loss = 152715.390 - ||grad||^2 = 89794.76638 - ||diff_w|| = 0.08168
 * Iteration #9 - Loss = 152699.873 - ||grad||^2 = 30787.72191 - ||diff_w|| = 0.03419
 * Iteration #10 - Loss = 152713.492 - ||grad||^2 = 10425.51700 - ||diff_w|| = 0.01469
 * Iteration #11 - Loss = 152729.801 - ||grad||^2 = 3631.35394 - ||diff_w|| = 0

LinearSVMModel

In [ ]:
c_index = concordance_index(svm_model, X_test, T_test, E_test) #0.93
print('C-index: {:f}'.format(c_index))

C-index: 0.829435


In [ ]:
from pysurvival.utils import save_model

In [ ]:
save_model(svm_model,'/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/SSVM/SSVM_new(onehot)_tgan-bal-40000.zip')

Saving the model to disk as /content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/SSVM/SSVM_new(onehot)_tgan-bal-40000.zip


/usr/local/lib/python3.7/dist-packages/pysurvival/utils/__init__.py:132: FutureWarning: 'pyarrow.serialize' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  model.save(path_file)


# tgan-bal-45000

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from pysurvival.models.svm import LinearSVMModel
from pysurvival.models.simulations import SimulationModel
from pysurvival.utils.metrics import concordance_index
from sklearn.model_selection import train_test_split
from scipy.stats.stats import pearsonr

In [ ]:
train_df = pd.read_csv('//content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/TGAN-bala-AR12/TGAN-syn-balanceAR-all-in-one-45000.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [ ]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
data_train = new_train_df
data_test = new_test_df

In [ ]:
sim = SimulationModel( survival_distribution = 'exponential',
                       risk_type = 'linear',
                       censored_parameter = 1,
                       alpha = 3)

In [ ]:
features = [f for f in data_train.columns if f not in ['a_year_time', 'a_year_event']]

time_column = 'a_year_time'
event_column = 'a_year_event'

In [ ]:
X_train, X_test = data_train[features], data_test[features]
T_train, T_test = data_train['a_year_time'].values, data_test['a_year_time'].values
E_train, E_test = data_train['a_year_event'].values, data_test['a_year_event'].values

In [ ]:
from sklearn.preprocessing import StandardScaler

X_scaler = StandardScaler().fit(X_train)
X_train = X_scaler.transform(X_train)
X_test = X_scaler.transform(X_test)

In [ ]:
svm_model = LinearSVMModel()
svm_model.fit(X_train, T_train, E_train, init_method='he_uniform',
    with_bias = True, lr = 0.5,  tol = 1e-3,  l2_reg = 1e-3)

Performing Newton-Raphson optimization: 
 * Iteration #1 - Loss = 29271655.605 - ||grad||^2 = 51648552.23965 - ||diff_w|| = 13.39003
 * Iteration #2 - Loss = 7421617.604 - ||grad||^2 = 26230286.62403 - ||diff_w|| = 6.01272
 * Iteration #3 - Loss = 2297956.826 - ||grad||^2 = 14706946.70873 - ||diff_w|| = 3.27013
 * Iteration #4 - Loss = 737798.459 - ||grad||^2 = 8099435.40354 - ||diff_w|| = 1.67426
 * Iteration #5 - Loss = 292883.635 - ||grad||^2 = 3966690.51166 - ||diff_w|| = 0.71689
 * Iteration #6 - Loss = 202056.518 - ||grad||^2 = 1923336.15156 - ||diff_w|| = 0.35075
 * Iteration #7 - Loss = 190619.342 - ||grad||^2 = 765375.64112 - ||diff_w|| = 0.17373
 * Iteration #8 - Loss = 191505.560 - ||grad||^2 = 247306.43291 - ||diff_w|| = 0.06917
 * Iteration #9 - Loss = 191599.638 - ||grad||^2 = 79983.35402 - ||diff_w|| = 0.02448
 * Iteration #10 - Loss = 191551.412 - ||grad||^2 = 26836.91132 - ||diff_w|| = 0.00975
 * Iteration #11 - Loss = 191545.970 - ||grad||^2 = 9360.67170 - ||diff_w|| 

LinearSVMModel

In [ ]:
c_index = concordance_index(svm_model, X_test, T_test, E_test) #0.93
print('C-index: {:f}'.format(c_index))

C-index: 0.829790


In [ ]:
from pysurvival.utils import save_model

In [ ]:
save_model(svm_model,'/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/SSVM/SSVM_new(onehot)_tgan-bal-45000.zip')

Saving the model to disk as /content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/SSVM/SSVM_new(onehot)_tgan-bal-45000.zip


/usr/local/lib/python3.7/dist-packages/pysurvival/utils/__init__.py:132: FutureWarning: 'pyarrow.serialize' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  model.save(path_file)


# tgan-bal-50000

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from pysurvival.models.svm import LinearSVMModel
from pysurvival.models.simulations import SimulationModel
from pysurvival.utils.metrics import concordance_index
from sklearn.model_selection import train_test_split
from scipy.stats.stats import pearsonr

In [ ]:
train_df = pd.read_csv('//content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/TGAN-bala-AR12/TGAN-syn-balanceAR-all-in-one-50000.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [ ]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
data_train = new_train_df
data_test = new_test_df

In [ ]:
sim = SimulationModel( survival_distribution = 'exponential',
                       risk_type = 'linear',
                       censored_parameter = 1,
                       alpha = 3)

In [ ]:
features = [f for f in data_train.columns if f not in ['a_year_time', 'a_year_event']]

time_column = 'a_year_time'
event_column = 'a_year_event'

In [ ]:
X_train, X_test = data_train[features], data_test[features]
T_train, T_test = data_train['a_year_time'].values, data_test['a_year_time'].values
E_train, E_test = data_train['a_year_event'].values, data_test['a_year_event'].values

In [ ]:
from sklearn.preprocessing import StandardScaler

X_scaler = StandardScaler().fit(X_train)
X_train = X_scaler.transform(X_train)
X_test = X_scaler.transform(X_test)

In [ ]:
svm_model = LinearSVMModel()
svm_model.fit(X_train, T_train, E_train, init_method='he_uniform',
    with_bias = True, lr = 0.5,  tol = 1e-3,  l2_reg = 1e-3)

Performing Newton-Raphson optimization: 
 * Iteration #1 - Loss = 37388534.606 - ||grad||^2 = 63507487.72153 - ||diff_w|| = 16.72109
 * Iteration #2 - Loss = 9436824.866 - ||grad||^2 = 32066802.31306 - ||diff_w|| = 7.51181
 * Iteration #3 - Loss = 2905903.893 - ||grad||^2 = 17834751.57784 - ||diff_w|| = 4.08526
 * Iteration #4 - Loss = 933161.497 - ||grad||^2 = 9778297.39047 - ||diff_w|| = 2.09211
 * Iteration #5 - Loss = 375208.001 - ||grad||^2 = 5025443.93784 - ||diff_w|| = 0.90811
 * Iteration #6 - Loss = 256635.751 - ||grad||^2 = 2392886.87875 - ||diff_w|| = 0.42782
 * Iteration #7 - Loss = 244224.086 - ||grad||^2 = 873692.07766 - ||diff_w|| = 0.20845
 * Iteration #8 - Loss = 245730.086 - ||grad||^2 = 244701.63054 - ||diff_w|| = 0.08681
 * Iteration #9 - Loss = 245822.611 - ||grad||^2 = 71240.47377 - ||diff_w|| = 0.03233
 * Iteration #10 - Loss = 245725.472 - ||grad||^2 = 23893.00154 - ||diff_w|| = 0.01367
 * Iteration #11 - Loss = 245700.945 - ||grad||^2 = 8560.82785 - ||diff_w|| 

LinearSVMModel

In [ ]:
c_index = concordance_index(svm_model, X_test, T_test, E_test) #0.93
print('C-index: {:f}'.format(c_index))

C-index: 0.829408


In [ ]:
from pysurvival.utils import save_model

In [ ]:
save_model(svm_model,'/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/SSVM/SSVM_new(onehot)_tgan-bal-50000.zip')

Saving the model to disk as /content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/SSVM/SSVM_new(onehot)_tgan-bal-50000.zip


/usr/local/lib/python3.7/dist-packages/pysurvival/utils/__init__.py:132: FutureWarning: 'pyarrow.serialize' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  model.save(path_file)


# tgan-aug-5000

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from pysurvival.models.svm import LinearSVMModel
from pysurvival.models.simulations import SimulationModel
from pysurvival.utils.metrics import concordance_index
from sklearn.model_selection import train_test_split
from scipy.stats.stats import pearsonr

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/TGAN-imbal-nonAR/TGAN-syn-imbar-nonAR-5000.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [ ]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
data_train = new_train_df
data_test = new_test_df

In [ ]:
sim = SimulationModel( survival_distribution = 'exponential',
                       risk_type = 'linear',
                       censored_parameter = 1,
                       alpha = 3)

In [ ]:
features = [f for f in data_train.columns if f not in ['a_year_time', 'a_year_event']]

time_column = 'a_year_time'
event_column = 'a_year_event'

In [ ]:
X_train, X_test = data_train[features], data_test[features]
T_train, T_test = data_train['a_year_time'].values, data_test['a_year_time'].values
E_train, E_test = data_train['a_year_event'].values, data_test['a_year_event'].values

In [ ]:
from sklearn.preprocessing import StandardScaler

X_scaler = StandardScaler().fit(X_train)
X_train = X_scaler.transform(X_train)
X_test = X_scaler.transform(X_test)

In [ ]:
svm_model = LinearSVMModel()
svm_model.fit(X_train, T_train, E_train, init_method='he_uniform',
    with_bias = True, lr = 0.5,  tol = 1e-3,  l2_reg = 1e-3)

Performing Newton-Raphson optimization: 
 * Iteration #1 - Loss = 108051.324 - ||grad||^2 = 138747.44533 - ||diff_w|| = 17.87421
 * Iteration #2 - Loss = 27448.960 - ||grad||^2 = 70218.09403 - ||diff_w|| = 8.03230
 * Iteration #3 - Loss = 8545.709 - ||grad||^2 = 39093.17899 - ||diff_w|| = 4.37546
 * Iteration #4 - Loss = 2821.681 - ||grad||^2 = 21159.53474 - ||diff_w|| = 2.24301
 * Iteration #5 - Loss = 1277.038 - ||grad||^2 = 9691.49468 - ||diff_w|| = 0.92314
 * Iteration #6 - Loss = 1014.076 - ||grad||^2 = 4027.32617 - ||diff_w|| = 0.37679
 * Iteration #7 - Loss = 987.692 - ||grad||^2 = 1810.27263 - ||diff_w|| = 0.18107
 * Iteration #8 - Loss = 986.160 - ||grad||^2 = 869.84435 - ||diff_w|| = 0.08875
 * Iteration #9 - Loss = 985.892 - ||grad||^2 = 416.78910 - ||diff_w|| = 0.04559
 * Iteration #10 - Loss = 985.746 - ||grad||^2 = 202.39371 - ||diff_w|| = 0.02428
 * Iteration #11 - Loss = 985.674 - ||grad||^2 = 101.36395 - ||diff_w|| = 0.01317
 * Iteration #12 - Loss = 985.659 - ||grad||

LinearSVMModel

In [ ]:
c_index = concordance_index(svm_model, X_test, T_test, E_test) #0.93
print('C-index: {:f}'.format(c_index))

C-index: 0.831346


In [ ]:
from pysurvival.utils import save_model

In [ ]:
save_model(svm_model,'/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/SSVM/SSVM_new(onehot)_tgan-aug-5000.zip')

Saving the model to disk as /content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/SSVM/SSVM_new(onehot)_tgan-aug-5000.zip


/usr/local/lib/python3.7/dist-packages/pysurvival/utils/__init__.py:132: FutureWarning: 'pyarrow.serialize' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  model.save(path_file)


# tgan-aug-10000

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from pysurvival.models.svm import LinearSVMModel
from pysurvival.models.simulations import SimulationModel
from pysurvival.utils.metrics import concordance_index
from sklearn.model_selection import train_test_split
from scipy.stats.stats import pearsonr

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/TGAN-imbal-nonAR/TGAN-syn-imbar-nonAR-10000.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [ ]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
data_train = new_train_df
data_test = new_test_df

In [ ]:
sim = SimulationModel( survival_distribution = 'exponential',
                       risk_type = 'linear',
                       censored_parameter = 1,
                       alpha = 3)

In [ ]:
features = [f for f in data_train.columns if f not in ['a_year_time', 'a_year_event']]

time_column = 'a_year_time'
event_column = 'a_year_event'

In [ ]:
X_train, X_test = data_train[features], data_test[features]
T_train, T_test = data_train['a_year_time'].values, data_test['a_year_time'].values
E_train, E_test = data_train['a_year_event'].values, data_test['a_year_event'].values

In [ ]:
from sklearn.preprocessing import StandardScaler

X_scaler = StandardScaler().fit(X_train)
X_train = X_scaler.transform(X_train)
X_test = X_scaler.transform(X_test)

In [ ]:
svm_model = LinearSVMModel()
svm_model.fit(X_train, T_train, E_train, init_method='he_uniform',
    with_bias = True, lr = 0.5,  tol = 1e-3,  l2_reg = 1e-3)

Performing Newton-Raphson optimization: 
 * Iteration #1 - Loss = 258437.854 - ||grad||^2 = 343936.11387 - ||diff_w|| = 18.48867
 * Iteration #2 - Loss = 67095.662 - ||grad||^2 = 178871.19314 - ||diff_w|| = 8.26919
 * Iteration #3 - Loss = 21951.716 - ||grad||^2 = 104994.03077 - ||diff_w|| = 4.40972
 * Iteration #4 - Loss = 8265.107 - ||grad||^2 = 59756.44459 - ||diff_w|| = 2.01575
 * Iteration #5 - Loss = 4641.990 - ||grad||^2 = 19662.52170 - ||diff_w|| = 0.66438
 * Iteration #6 - Loss = 4141.265 - ||grad||^2 = 7749.60618 - ||diff_w|| = 0.27543
 * Iteration #7 - Loss = 4132.216 - ||grad||^2 = 2716.50659 - ||diff_w|| = 0.12504
 * Iteration #8 - Loss = 4135.666 - ||grad||^2 = 1023.00347 - ||diff_w|| = 0.05933
 * Iteration #9 - Loss = 4136.408 - ||grad||^2 = 402.72138 - ||diff_w|| = 0.02903
 * Iteration #10 - Loss = 4136.621 - ||grad||^2 = 168.22754 - ||diff_w|| = 0.01488
 * Iteration #11 - Loss = 4136.686 - ||grad||^2 = 77.16438 - ||diff_w|| = 0.00784
 * Iteration #12 - Loss = 4136.713 

LinearSVMModel

In [ ]:
c_index = concordance_index(svm_model, X_test, T_test, E_test) #0.93
print('C-index: {:f}'.format(c_index))

C-index: 0.827587


In [ ]:
from pysurvival.utils import save_model

In [ ]:
save_model(svm_model,'/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/SSVM/SSVM_new(onehot)_tgan-aug-10000.zip')

Saving the model to disk as /content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/SSVM/SSVM_new(onehot)_tgan-aug-10000.zip


/usr/local/lib/python3.7/dist-packages/pysurvival/utils/__init__.py:132: FutureWarning: 'pyarrow.serialize' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  model.save(path_file)


# tgan-aug-15000

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from pysurvival.models.svm import LinearSVMModel
from pysurvival.models.simulations import SimulationModel
from pysurvival.utils.metrics import concordance_index
from sklearn.model_selection import train_test_split
from scipy.stats.stats import pearsonr

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/TGAN-imbal-nonAR/TGAN-syn-imbar-nonAR-15000.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [ ]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
data_train = new_train_df
data_test = new_test_df

In [ ]:
sim = SimulationModel( survival_distribution = 'exponential',
                       risk_type = 'linear',
                       censored_parameter = 1,
                       alpha = 3)

In [ ]:
features = [f for f in data_train.columns if f not in ['a_year_time', 'a_year_event']]

time_column = 'a_year_time'
event_column = 'a_year_event'

In [ ]:
X_train, X_test = data_train[features], data_test[features]
T_train, T_test = data_train['a_year_time'].values, data_test['a_year_time'].values
E_train, E_test = data_train['a_year_event'].values, data_test['a_year_event'].values

In [ ]:
from sklearn.preprocessing import StandardScaler

X_scaler = StandardScaler().fit(X_train)
X_train = X_scaler.transform(X_train)
X_test = X_scaler.transform(X_test)

In [ ]:
svm_model = LinearSVMModel()
svm_model.fit(X_train, T_train, E_train, init_method='he_uniform',
    with_bias = True, lr = 0.5,  tol = 1e-3,  l2_reg = 1e-3)

Performing Newton-Raphson optimization: 
 * Iteration #1 - Loss = 1015720.745 - ||grad||^2 = 1741711.96236 - ||diff_w|| = 17.00016
 * Iteration #2 - Loss = 258559.911 - ||grad||^2 = 881368.78134 - ||diff_w|| = 7.62858
 * Iteration #3 - Loss = 81087.492 - ||grad||^2 = 491242.88535 - ||diff_w|| = 4.13191
 * Iteration #4 - Loss = 27297.142 - ||grad||^2 = 266706.44836 - ||diff_w|| = 2.06114
 * Iteration #5 - Loss = 12278.481 - ||grad||^2 = 130859.17533 - ||diff_w|| = 0.77699
 * Iteration #6 - Loss = 9331.209 - ||grad||^2 = 61079.11173 - ||diff_w|| = 0.31418
 * Iteration #7 - Loss = 9030.490 - ||grad||^2 = 24050.69577 - ||diff_w|| = 0.13314
 * Iteration #8 - Loss = 9061.119 - ||grad||^2 = 7943.67932 - ||diff_w|| = 0.05043
 * Iteration #9 - Loss = 9073.467 - ||grad||^2 = 2694.18647 - ||diff_w|| = 0.01980
 * Iteration #10 - Loss = 9075.698 - ||grad||^2 = 1028.75785 - ||diff_w|| = 0.00847
 * Iteration #11 - Loss = 9076.006 - ||grad||^2 = 432.68061 - ||diff_w|| = 0.00386
 * Iteration #12 - Loss

LinearSVMModel

In [ ]:
c_index = concordance_index(svm_model, X_test, T_test, E_test) #0.93
print('C-index: {:f}'.format(c_index))

C-index: 0.828600


In [ ]:
from pysurvival.utils import save_model

In [ ]:
save_model(svm_model,'/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/SSVM/SSVM_new(onehot)_tgan-aug-15000.zip')

Saving the model to disk as /content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/SSVM/SSVM_new(onehot)_tgan-aug-15000.zip


/usr/local/lib/python3.7/dist-packages/pysurvival/utils/__init__.py:132: FutureWarning: 'pyarrow.serialize' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  model.save(path_file)


# tgan-aug-20000

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from pysurvival.models.svm import LinearSVMModel
from pysurvival.models.simulations import SimulationModel
from pysurvival.utils.metrics import concordance_index
from sklearn.model_selection import train_test_split
from scipy.stats.stats import pearsonr

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/TGAN-imbal-nonAR/TGAN-syn-imbar-nonAR-20000.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [ ]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
data_train = new_train_df
data_test = new_test_df

In [ ]:
sim = SimulationModel( survival_distribution = 'exponential',
                       risk_type = 'linear',
                       censored_parameter = 1,
                       alpha = 3)

In [ ]:
features = [f for f in data_train.columns if f not in ['a_year_time', 'a_year_event']]

time_column = 'a_year_time'
event_column = 'a_year_event'

In [ ]:
X_train, X_test = data_train[features], data_test[features]
T_train, T_test = data_train['a_year_time'].values, data_test['a_year_time'].values
E_train, E_test = data_train['a_year_event'].values, data_test['a_year_event'].values

In [ ]:
from sklearn.preprocessing import StandardScaler

X_scaler = StandardScaler().fit(X_train)
X_train = X_scaler.transform(X_train)
X_test = X_scaler.transform(X_test)

In [ ]:
svm_model = LinearSVMModel()
svm_model.fit(X_train, T_train, E_train, init_method='he_uniform',
    with_bias = True, lr = 0.5,  tol = 1e-3,  l2_reg = 1e-3)

Performing Newton-Raphson optimization: 
 * Iteration #1 - Loss = 1672691.680 - ||grad||^2 = 2626335.64482 - ||diff_w|| = 14.07636
 * Iteration #2 - Loss = 424439.007 - ||grad||^2 = 1330783.75637 - ||diff_w|| = 6.30803
 * Iteration #3 - Loss = 132247.972 - ||grad||^2 = 741651.48288 - ||diff_w|| = 3.40006
 * Iteration #4 - Loss = 44314.124 - ||grad||^2 = 398363.29280 - ||diff_w|| = 1.68447
 * Iteration #5 - Loss = 20864.771 - ||grad||^2 = 193664.99718 - ||diff_w|| = 0.71976
 * Iteration #6 - Loss = 16438.440 - ||grad||^2 = 88538.27327 - ||diff_w|| = 0.32532
 * Iteration #7 - Loss = 16063.049 - ||grad||^2 = 35922.77545 - ||diff_w|| = 0.13983
 * Iteration #8 - Loss = 16120.709 - ||grad||^2 = 12658.87497 - ||diff_w|| = 0.05803
 * Iteration #9 - Loss = 16138.362 - ||grad||^2 = 4580.12904 - ||diff_w|| = 0.02640
 * Iteration #10 - Loss = 16141.674 - ||grad||^2 = 1757.04556 - ||diff_w|| = 0.01278
 * Iteration #11 - Loss = 16142.257 - ||grad||^2 = 715.86215 - ||diff_w|| = 0.00640
 * Iteration #

LinearSVMModel

In [ ]:
c_index = concordance_index(svm_model, X_test, T_test, E_test) #0.93
print('C-index: {:f}'.format(c_index))

C-index: 0.829053


In [ ]:
from pysurvival.utils import save_model

In [ ]:
save_model(svm_model,'/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/SSVM/SSVM_new(onehot)_tgan-aug-20000.zip')

Saving the model to disk as /content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/SSVM/SSVM_new(onehot)_tgan-aug-20000.zip


/usr/local/lib/python3.7/dist-packages/pysurvival/utils/__init__.py:132: FutureWarning: 'pyarrow.serialize' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  model.save(path_file)


# tgan-aug-25000

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from pysurvival.models.svm import LinearSVMModel
from pysurvival.models.simulations import SimulationModel
from pysurvival.utils.metrics import concordance_index
from sklearn.model_selection import train_test_split
from scipy.stats.stats import pearsonr

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/TGAN-imbal-nonAR/TGAN-syn-imbar-nonAR-25000.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [ ]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
data_train = new_train_df
data_test = new_test_df

In [ ]:
sim = SimulationModel( survival_distribution = 'exponential',
                       risk_type = 'linear',
                       censored_parameter = 1,
                       alpha = 3)

In [ ]:
features = [f for f in data_train.columns if f not in ['a_year_time', 'a_year_event']]

time_column = 'a_year_time'
event_column = 'a_year_event'

In [ ]:
X_train, X_test = data_train[features], data_test[features]
T_train, T_test = data_train['a_year_time'].values, data_test['a_year_time'].values
E_train, E_test = data_train['a_year_event'].values, data_test['a_year_event'].values

In [ ]:
from sklearn.preprocessing import StandardScaler

X_scaler = StandardScaler().fit(X_train)
X_train = X_scaler.transform(X_train)
X_test = X_scaler.transform(X_test)

In [ ]:
svm_model = LinearSVMModel()
svm_model.fit(X_train, T_train, E_train, init_method='he_uniform',
    with_bias = True, lr = 0.5,  tol = 1e-3,  l2_reg = 1e-3)

Performing Newton-Raphson optimization: 
 * Iteration #1 - Loss = 5126602.510 - ||grad||^2 = 6875661.80401 - ||diff_w|| = 19.36605
 * Iteration #2 - Loss = 1295422.764 - ||grad||^2 = 3484714.43881 - ||diff_w|| = 8.69887
 * Iteration #3 - Loss = 397714.326 - ||grad||^2 = 1950927.72513 - ||diff_w|| = 4.74391
 * Iteration #4 - Loss = 124323.492 - ||grad||^2 = 1084421.79927 - ||diff_w|| = 2.48604
 * Iteration #5 - Loss = 45033.209 - ||grad||^2 = 566113.62576 - ||diff_w|| = 1.11842
 * Iteration #6 - Loss = 27223.249 - ||grad||^2 = 266649.13867 - ||diff_w|| = 0.47299
 * Iteration #7 - Loss = 24820.293 - ||grad||^2 = 106433.17853 - ||diff_w|| = 0.22254
 * Iteration #8 - Loss = 24857.896 - ||grad||^2 = 38548.86699 - ||diff_w|| = 0.09628
 * Iteration #9 - Loss = 24917.333 - ||grad||^2 = 16023.62590 - ||diff_w|| = 0.03955
 * Iteration #10 - Loss = 24921.042 - ||grad||^2 = 7316.47099 - ||diff_w|| = 0.01826
 * Iteration #11 - Loss = 24919.854 - ||grad||^2 = 3561.48196 - ||diff_w|| = 0.00904
 * Ite

LinearSVMModel

In [ ]:
c_index = concordance_index(svm_model, X_test, T_test, E_test) #0.93
print('C-index: {:f}'.format(c_index))

C-index: 0.828902


In [ ]:
from pysurvival.utils import save_model

In [ ]:
save_model(svm_model,'/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/SSVM/SSVM_new(onehot)_tgan-aug-25000.zip')

Saving the model to disk as /content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/SSVM/SSVM_new(onehot)_tgan-aug-25000.zip


/usr/local/lib/python3.7/dist-packages/pysurvival/utils/__init__.py:132: FutureWarning: 'pyarrow.serialize' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  model.save(path_file)


# tgan-aug-30000

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from pysurvival.models.svm import LinearSVMModel
from pysurvival.models.simulations import SimulationModel
from pysurvival.utils.metrics import concordance_index
from sklearn.model_selection import train_test_split
from scipy.stats.stats import pearsonr

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/TGAN-imbal-nonAR/TGAN-syn-imbar-nonAR-30000.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [ ]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
data_train = new_train_df
data_test = new_test_df

In [ ]:
sim = SimulationModel( survival_distribution = 'exponential',
                       risk_type = 'linear',
                       censored_parameter = 1,
                       alpha = 3)

In [ ]:
features = [f for f in data_train.columns if f not in ['a_year_time', 'a_year_event']]

time_column = 'a_year_time'
event_column = 'a_year_event'

In [ ]:
X_train, X_test = data_train[features], data_test[features]
T_train, T_test = data_train['a_year_time'].values, data_test['a_year_time'].values
E_train, E_test = data_train['a_year_event'].values, data_test['a_year_event'].values

In [ ]:
from sklearn.preprocessing import StandardScaler

X_scaler = StandardScaler().fit(X_train)
X_train = X_scaler.transform(X_train)
X_test = X_scaler.transform(X_test)

In [ ]:
svm_model = LinearSVMModel()
svm_model.fit(X_train, T_train, E_train, init_method='he_uniform',
    with_bias = True, lr = 0.5,  tol = 1e-3,  l2_reg = 1e-3)

Performing Newton-Raphson optimization: 
 * Iteration #1 - Loss = 3878208.837 - ||grad||^2 = 5519710.00138 - ||diff_w|| = 17.51438
 * Iteration #2 - Loss = 992759.802 - ||grad||^2 = 2818569.60113 - ||diff_w|| = 7.86127
 * Iteration #3 - Loss = 313681.873 - ||grad||^2 = 1606249.86804 - ||diff_w|| = 4.26734
 * Iteration #4 - Loss = 105770.749 - ||grad||^2 = 907484.48217 - ||diff_w|| = 2.15466
 * Iteration #5 - Loss = 45823.196 - ||grad||^2 = 413087.25005 - ||diff_w|| = 0.82952
 * Iteration #6 - Loss = 35191.767 - ||grad||^2 = 150330.86130 - ||diff_w|| = 0.32610
 * Iteration #7 - Loss = 35259.060 - ||grad||^2 = 45940.80580 - ||diff_w|| = 0.14203
 * Iteration #8 - Loss = 35313.416 - ||grad||^2 = 15318.98041 - ||diff_w|| = 0.05917
 * Iteration #9 - Loss = 35318.030 - ||grad||^2 = 5733.00254 - ||diff_w|| = 0.02598
 * Iteration #10 - Loss = 35318.522 - ||grad||^2 = 2260.95742 - ||diff_w|| = 0.01233
 * Iteration #11 - Loss = 35318.532 - ||grad||^2 = 917.28641 - ||diff_w|| = 0.00610
 * Iteratio

LinearSVMModel

In [ ]:
c_index = concordance_index(svm_model, X_test, T_test, E_test) #0.93
print('C-index: {:f}'.format(c_index))

C-index: 0.829702


In [ ]:
from pysurvival.utils import save_model

In [ ]:
save_model(svm_model,'/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/SSVM/SSVM_new(onehot)_tgan-aug-30000.zip')

Saving the model to disk as /content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/SSVM/SSVM_new(onehot)_tgan-aug-30000.zip


/usr/local/lib/python3.7/dist-packages/pysurvival/utils/__init__.py:132: FutureWarning: 'pyarrow.serialize' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  model.save(path_file)


# tgan-aug-35000

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from pysurvival.models.svm import LinearSVMModel
from pysurvival.models.simulations import SimulationModel
from pysurvival.utils.metrics import concordance_index
from sklearn.model_selection import train_test_split
from scipy.stats.stats import pearsonr

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/TGAN-imbal-nonAR/TGAN-syn-imbar-nonAR-35000.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [ ]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
data_train = new_train_df
data_test = new_test_df

In [ ]:
sim = SimulationModel( survival_distribution = 'exponential',
                       risk_type = 'linear',
                       censored_parameter = 1,
                       alpha = 3)

In [ ]:
features = [f for f in data_train.columns if f not in ['a_year_time', 'a_year_event']]

time_column = 'a_year_time'
event_column = 'a_year_event'

In [ ]:
X_train, X_test = data_train[features], data_test[features]
T_train, T_test = data_train['a_year_time'].values, data_test['a_year_time'].values
E_train, E_test = data_train['a_year_event'].values, data_test['a_year_event'].values

In [ ]:
from sklearn.preprocessing import StandardScaler

X_scaler = StandardScaler().fit(X_train)
X_train = X_scaler.transform(X_train)
X_test = X_scaler.transform(X_test)

In [ ]:
svm_model = LinearSVMModel()
svm_model.fit(X_train, T_train, E_train, init_method='he_uniform',
    with_bias = True, lr = 0.5,  tol = 1e-3,  l2_reg = 1e-3)

Performing Newton-Raphson optimization: 
 * Iteration #1 - Loss = 7599275.567 - ||grad||^2 = 11871713.02894 - ||diff_w|| = 14.88904
 * Iteration #2 - Loss = 1922508.074 - ||grad||^2 = 6007609.38805 - ||diff_w|| = 6.68104
 * Iteration #3 - Loss = 592763.983 - ||grad||^2 = 3351742.91713 - ||diff_w|| = 3.62653
 * Iteration #4 - Loss = 189165.259 - ||grad||^2 = 1845298.03442 - ||diff_w|| = 1.86048
 * Iteration #5 - Loss = 75337.262 - ||grad||^2 = 958113.09306 - ||diff_w|| = 0.81697
 * Iteration #6 - Loss = 51380.390 - ||grad||^2 = 448800.19958 - ||diff_w|| = 0.36868
 * Iteration #7 - Loss = 48602.881 - ||grad||^2 = 170317.34228 - ||diff_w|| = 0.16703
 * Iteration #8 - Loss = 48909.786 - ||grad||^2 = 61098.69898 - ||diff_w|| = 0.06427
 * Iteration #9 - Loss = 48989.676 - ||grad||^2 = 25265.11260 - ||diff_w|| = 0.02632
 * Iteration #10 - Loss = 48992.175 - ||grad||^2 = 10643.13011 - ||diff_w|| = 0.01211
 * Iteration #11 - Loss = 48991.415 - ||grad||^2 = 4774.98284 - ||diff_w|| = 0.00601
 * I

LinearSVMModel

In [ ]:
c_index = concordance_index(svm_model, X_test, T_test, E_test) #0.93
print('C-index: {:f}'.format(c_index))

C-index: 0.831275


In [ ]:
from pysurvival.utils import save_model

In [ ]:
save_model(svm_model,'/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/SSVM/SSVM_new(onehot)_tgan-aug-35000.zip')

Saving the model to disk as /content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/SSVM/SSVM_new(onehot)_tgan-aug-35000.zip


/usr/local/lib/python3.7/dist-packages/pysurvival/utils/__init__.py:132: FutureWarning: 'pyarrow.serialize' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  model.save(path_file)


# tgan-aug-40000

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from pysurvival.models.svm import LinearSVMModel
from pysurvival.models.simulations import SimulationModel
from pysurvival.utils.metrics import concordance_index
from sklearn.model_selection import train_test_split
from scipy.stats.stats import pearsonr

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/TGAN-imbal-nonAR/TGAN-syn-imbar-nonAR-40000.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [ ]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
data_train = new_train_df
data_test = new_test_df

In [ ]:
sim = SimulationModel( survival_distribution = 'exponential',
                       risk_type = 'linear',
                       censored_parameter = 1,
                       alpha = 3)

In [ ]:
features = [f for f in data_train.columns if f not in ['a_year_time', 'a_year_event']]

time_column = 'a_year_time'
event_column = 'a_year_event'

In [ ]:
X_train, X_test = data_train[features], data_test[features]
T_train, T_test = data_train['a_year_time'].values, data_test['a_year_time'].values
E_train, E_test = data_train['a_year_event'].values, data_test['a_year_event'].values

In [ ]:
from sklearn.preprocessing import StandardScaler

X_scaler = StandardScaler().fit(X_train)
X_train = X_scaler.transform(X_train)
X_test = X_scaler.transform(X_test)

In [ ]:
svm_model = LinearSVMModel()
svm_model.fit(X_train, T_train, E_train, init_method='he_uniform',
    with_bias = True, lr = 0.5,  tol = 1e-3,  l2_reg = 1e-3)

Performing Newton-Raphson optimization: 
 * Iteration #1 - Loss = 5369581.148 - ||grad||^2 = 9453948.89009 - ||diff_w|| = 17.62188
 * Iteration #2 - Loss = 1372424.893 - ||grad||^2 = 4778937.86501 - ||diff_w|| = 7.89496
 * Iteration #3 - Loss = 435408.798 - ||grad||^2 = 2648818.21589 - ||diff_w|| = 4.23658
 * Iteration #4 - Loss = 153379.889 - ||grad||^2 = 1402306.19971 - ||diff_w|| = 2.00184
 * Iteration #5 - Loss = 78976.090 - ||grad||^2 = 666331.17235 - ||diff_w|| = 0.69830
 * Iteration #6 - Loss = 67036.390 - ||grad||^2 = 299339.19177 - ||diff_w|| = 0.28310
 * Iteration #7 - Loss = 66012.057 - ||grad||^2 = 116021.09687 - ||diff_w|| = 0.12546
 * Iteration #8 - Loss = 66050.343 - ||grad||^2 = 43563.87966 - ||diff_w|| = 0.05430
 * Iteration #9 - Loss = 66068.685 - ||grad||^2 = 18982.19760 - ||diff_w|| = 0.02477
 * Iteration #10 - Loss = 66071.172 - ||grad||^2 = 8966.82405 - ||diff_w|| = 0.01210
 * Iteration #11 - Loss = 66070.965 - ||grad||^2 = 4449.25235 - ||diff_w|| = 0.00623
 * Ite

LinearSVMModel

In [ ]:
c_index = concordance_index(svm_model, X_test, T_test, E_test) #0.93
print('C-index: {:f}'.format(c_index))

C-index: 0.829755


In [ ]:
from pysurvival.utils import save_model

In [ ]:
save_model(svm_model,'/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/SSVM/SSVM_new(onehot)_tgan-aug-40000.zip')

Saving the model to disk as /content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/SSVM/SSVM_new(onehot)_tgan-aug-40000.zip


/usr/local/lib/python3.7/dist-packages/pysurvival/utils/__init__.py:132: FutureWarning: 'pyarrow.serialize' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  model.save(path_file)


# tgan-aug-45000

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from pysurvival.models.svm import LinearSVMModel
from pysurvival.models.simulations import SimulationModel
from pysurvival.utils.metrics import concordance_index
from sklearn.model_selection import train_test_split
from scipy.stats.stats import pearsonr

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/TGAN-imbal-nonAR/TGAN-syn-imbar-nonAR-45000.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [ ]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
data_train = new_train_df
data_test = new_test_df

In [ ]:
sim = SimulationModel( survival_distribution = 'exponential',
                       risk_type = 'linear',
                       censored_parameter = 1,
                       alpha = 3)

In [ ]:
features = [f for f in data_train.columns if f not in ['a_year_time', 'a_year_event']]

time_column = 'a_year_time'
event_column = 'a_year_event'

In [ ]:
X_train, X_test = data_train[features], data_test[features]
T_train, T_test = data_train['a_year_time'].values, data_test['a_year_time'].values
E_train, E_test = data_train['a_year_event'].values, data_test['a_year_event'].values

In [ ]:
from sklearn.preprocessing import StandardScaler

X_scaler = StandardScaler().fit(X_train)
X_train = X_scaler.transform(X_train)
X_test = X_scaler.transform(X_test)

In [ ]:
svm_model = LinearSVMModel()
svm_model.fit(X_train, T_train, E_train, init_method='he_uniform',
    with_bias = True, lr = 0.5,  tol = 1e-3,  l2_reg = 1e-3)

Performing Newton-Raphson optimization: 
 * Iteration #1 - Loss = 5653390.523 - ||grad||^2 = 9616056.56523 - ||diff_w|| = 14.77140
 * Iteration #2 - Loss = 1461728.763 - ||grad||^2 = 4926067.99497 - ||diff_w|| = 6.60955
 * Iteration #3 - Loss = 473684.687 - ||grad||^2 = 2835497.43138 - ||diff_w|| = 3.53211
 * Iteration #4 - Loss = 173429.079 - ||grad||^2 = 1576100.01208 - ||diff_w|| = 1.63191
 * Iteration #5 - Loss = 93373.863 - ||grad||^2 = 568960.10290 - ||diff_w|| = 0.52262
 * Iteration #6 - Loss = 82128.746 - ||grad||^2 = 199203.76592 - ||diff_w|| = 0.18862
 * Iteration #7 - Loss = 81646.424 - ||grad||^2 = 69243.15675 - ||diff_w|| = 0.08380
 * Iteration #8 - Loss = 81659.211 - ||grad||^2 = 25574.56424 - ||diff_w|| = 0.03909
 * Iteration #9 - Loss = 81661.930 - ||grad||^2 = 11229.67338 - ||diff_w|| = 0.01879
 * Iteration #10 - Loss = 81662.404 - ||grad||^2 = 5390.38876 - ||diff_w|| = 0.00960
 * Iteration #11 - Loss = 81662.579 - ||grad||^2 = 2760.65807 - ||diff_w|| = 0.00502
 * Iter

LinearSVMModel

In [ ]:
c_index = concordance_index(svm_model, X_test, T_test, E_test) #0.93
print('C-index: {:f}'.format(c_index))

C-index: 0.831461


In [ ]:
from pysurvival.utils import save_model

In [ ]:
save_model(svm_model,'/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/SSVM/SSVM_new(onehot)_tgan-aug-45000.zip')

Saving the model to disk as /content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/SSVM/SSVM_new(onehot)_tgan-aug-45000.zip


/usr/local/lib/python3.7/dist-packages/pysurvival/utils/__init__.py:132: FutureWarning: 'pyarrow.serialize' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  model.save(path_file)


# tgan-aug-50000

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from pysurvival.models.svm import LinearSVMModel
from pysurvival.models.simulations import SimulationModel
from pysurvival.utils.metrics import concordance_index
from sklearn.model_selection import train_test_split
from scipy.stats.stats import pearsonr

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/TGAN-imbal-nonAR/TGAN-syn-imbar-nonAR-50000.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [ ]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
data_train = new_train_df
data_test = new_test_df

In [ ]:
sim = SimulationModel( survival_distribution = 'exponential',
                       risk_type = 'linear',
                       censored_parameter = 1,
                       alpha = 3)

In [ ]:
features = [f for f in data_train.columns if f not in ['a_year_time', 'a_year_event']]

time_column = 'a_year_time'
event_column = 'a_year_event'

In [ ]:
X_train, X_test = data_train[features], data_test[features]
T_train, T_test = data_train['a_year_time'].values, data_test['a_year_time'].values
E_train, E_test = data_train['a_year_event'].values, data_test['a_year_event'].values

In [ ]:
from sklearn.preprocessing import StandardScaler

X_scaler = StandardScaler().fit(X_train)
X_train = X_scaler.transform(X_train)
X_test = X_scaler.transform(X_test)

In [ ]:
svm_model = LinearSVMModel()
svm_model.fit(X_train, T_train, E_train, init_method='he_uniform',
    with_bias = True, lr = 0.5,  tol = 1e-3,  l2_reg = 1e-3)

Performing Newton-Raphson optimization: 
 * Iteration #1 - Loss = 8277014.253 - ||grad||^2 = 13726613.41928 - ||diff_w|| = 16.65035
 * Iteration #2 - Loss = 2102260.238 - ||grad||^2 = 6950792.68063 - ||diff_w|| = 7.45346
 * Iteration #3 - Loss = 658163.289 - ||grad||^2 = 3851769.41402 - ||diff_w|| = 3.98135
 * Iteration #4 - Loss = 228661.930 - ||grad||^2 = 2043950.42605 - ||diff_w|| = 1.88199
 * Iteration #5 - Loss = 121174.255 - ||grad||^2 = 1028568.53788 - ||diff_w|| = 0.80917
 * Iteration #6 - Loss = 102143.815 - ||grad||^2 = 463588.70096 - ||diff_w|| = 0.38495
 * Iteration #7 - Loss = 100565.546 - ||grad||^2 = 174604.48494 - ||diff_w|| = 0.16950
 * Iteration #8 - Loss = 100768.598 - ||grad||^2 = 65683.35225 - ||diff_w|| = 0.06776
 * Iteration #9 - Loss = 100854.461 - ||grad||^2 = 25211.71147 - ||diff_w|| = 0.02758
 * Iteration #10 - Loss = 100876.678 - ||grad||^2 = 10229.99415 - ||diff_w|| = 0.01196
 * Iteration #11 - Loss = 100883.418 - ||grad||^2 = 4570.17278 - ||diff_w|| = 0.00

LinearSVMModel

In [ ]:
c_index = concordance_index(svm_model, X_test, T_test, E_test) #0.93
print('C-index: {:f}'.format(c_index))

C-index: 0.829862


In [ ]:
from pysurvival.utils import save_model

In [ ]:
save_model(svm_model,'/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/SSVM/SSVM_new(onehot)_tgan-aug-50000.zip')

Saving the model to disk as /content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/SSVM/SSVM_new(onehot)_tgan-aug-50000.zip


/usr/local/lib/python3.7/dist-packages/pysurvival/utils/__init__.py:132: FutureWarning: 'pyarrow.serialize' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  model.save(path_file)


# tgan-org+balAR-6616

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from pysurvival.models.svm import LinearSVMModel
from pysurvival.models.simulations import SimulationModel
from pysurvival.utils.metrics import concordance_index
from sklearn.model_selection import train_test_split
from scipy.stats.stats import pearsonr

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/original+TGAN-balAR/origin+tganbalAR-6616.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [ ]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
data_train = new_train_df
data_test = new_test_df

In [ ]:
sim = SimulationModel( survival_distribution = 'exponential',
                       risk_type = 'linear',
                       censored_parameter = 1,
                       alpha = 3)

In [ ]:
features = [f for f in data_train.columns if f not in ['a_year_time', 'a_year_event']]

time_column = 'a_year_time'
event_column = 'a_year_event'

In [ ]:
X_train, X_test = data_train[features], data_test[features]
T_train, T_test = data_train['a_year_time'].values, data_test['a_year_time'].values
E_train, E_test = data_train['a_year_event'].values, data_test['a_year_event'].values

In [ ]:
from sklearn.preprocessing import StandardScaler

X_scaler = StandardScaler().fit(X_train)
X_train = X_scaler.transform(X_train)
X_test = X_scaler.transform(X_test)

In [ ]:
svm_model = LinearSVMModel()
svm_model.fit(X_train, T_train, E_train, init_method='he_uniform',
    with_bias = True, lr = 0.5,  tol = 1e-3,  l2_reg = 1e-3)

Performing Newton-Raphson optimization: 
 * Iteration #1 - Loss = 240913.415 - ||grad||^2 = 324735.96632 - ||diff_w|| = 16.12766
 * Iteration #2 - Loss = 61842.311 - ||grad||^2 = 165660.53742 - ||diff_w|| = 7.20866
 * Iteration #3 - Loss = 19788.100 - ||grad||^2 = 93331.82783 - ||diff_w|| = 3.82949
 * Iteration #4 - Loss = 7203.968 - ||grad||^2 = 49383.52452 - ||diff_w|| = 1.72154
 * Iteration #5 - Loss = 4073.801 - ||grad||^2 = 21768.48682 - ||diff_w|| = 0.59772
 * Iteration #6 - Loss = 3691.525 - ||grad||^2 = 8360.84445 - ||diff_w|| = 0.24208
 * Iteration #7 - Loss = 3686.440 - ||grad||^2 = 3036.53618 - ||diff_w|| = 0.09609
 * Iteration #8 - Loss = 3684.498 - ||grad||^2 = 1162.42334 - ||diff_w|| = 0.03869
 * Iteration #9 - Loss = 3684.232 - ||grad||^2 = 454.61920 - ||diff_w|| = 0.01609
 * Iteration #10 - Loss = 3684.363 - ||grad||^2 = 186.50696 - ||diff_w|| = 0.00703
 * Iteration #11 - Loss = 3684.381 - ||grad||^2 = 77.70458 - ||diff_w|| = 0.00318
 * Iteration #12 - Loss = 3684.373 -

LinearSVMModel

In [ ]:
c_index = concordance_index(svm_model, X_test, T_test, E_test) #0.93
print('C-index: {:f}'.format(c_index))

C-index: 0.834963


In [ ]:
from pysurvival.utils import save_model

In [ ]:
save_model(svm_model,'/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/SSVM/SSVM_new(onehot)_tgan-org+balAR-6616.zip')

Saving the model to disk as /content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/SSVM/SSVM_new(onehot)_tgan-org+balAR-6616.zip


/usr/local/lib/python3.7/dist-packages/pysurvival/utils/__init__.py:132: FutureWarning: 'pyarrow.serialize' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  model.save(path_file)


# tgan-org+balAR-11616

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from pysurvival.models.svm import LinearSVMModel
from pysurvival.models.simulations import SimulationModel
from pysurvival.utils.metrics import concordance_index
from sklearn.model_selection import train_test_split
from scipy.stats.stats import pearsonr

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/original+TGAN-balAR/origin+tganbalAR-11616.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [ ]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
data_train = new_train_df
data_test = new_test_df

In [ ]:
sim = SimulationModel( survival_distribution = 'exponential',
                       risk_type = 'linear',
                       censored_parameter = 1,
                       alpha = 3)

In [ ]:
features = [f for f in data_train.columns if f not in ['a_year_time', 'a_year_event']]

time_column = 'a_year_time'
event_column = 'a_year_event'

In [ ]:
X_train, X_test = data_train[features], data_test[features]
T_train, T_test = data_train['a_year_time'].values, data_test['a_year_time'].values
E_train, E_test = data_train['a_year_event'].values, data_test['a_year_event'].values

In [ ]:
from sklearn.preprocessing import StandardScaler

X_scaler = StandardScaler().fit(X_train)
X_train = X_scaler.transform(X_train)
X_test = X_scaler.transform(X_test)

In [ ]:
svm_model = LinearSVMModel()
svm_model.fit(X_train, T_train, E_train, init_method='he_uniform',
    with_bias = True, lr = 0.5,  tol = 1e-3,  l2_reg = 1e-3)

Performing Newton-Raphson optimization: 
 * Iteration #1 - Loss = 2793116.558 - ||grad||^2 = 4461179.73052 - ||diff_w|| = 14.96894
 * Iteration #2 - Loss = 709704.281 - ||grad||^2 = 2271999.11641 - ||diff_w|| = 6.71699
 * Iteration #3 - Loss = 219518.408 - ||grad||^2 = 1280860.13079 - ||diff_w|| = 3.65574
 * Iteration #4 - Loss = 68867.040 - ||grad||^2 = 718303.57395 - ||diff_w|| = 1.92967
 * Iteration #5 - Loss = 23982.187 - ||grad||^2 = 374375.09011 - ||diff_w|| = 0.93097
 * Iteration #6 - Loss = 13147.867 - ||grad||^2 = 186247.03533 - ||diff_w|| = 0.44273
 * Iteration #7 - Loss = 11552.535 - ||grad||^2 = 70504.85502 - ||diff_w|| = 0.21196
 * Iteration #8 - Loss = 11661.293 - ||grad||^2 = 21104.85493 - ||diff_w|| = 0.08362
 * Iteration #9 - Loss = 11646.070 - ||grad||^2 = 7137.69138 - ||diff_w|| = 0.03092
 * Iteration #10 - Loss = 11643.111 - ||grad||^2 = 2474.59546 - ||diff_w|| = 0.01260
 * Iteration #11 - Loss = 11643.642 - ||grad||^2 = 906.59412 - ||diff_w|| = 0.00562
 * Iteration

LinearSVMModel

In [ ]:
c_index = concordance_index(svm_model, X_test, T_test, E_test) #0.93
print('C-index: {:f}'.format(c_index))

C-index: 0.832394


In [ ]:
from pysurvival.utils import save_model

In [ ]:
save_model(svm_model,'/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/SSVM/SSVM_new(onehot)_tgan-org+balAR-11616.zip')

Saving the model to disk as /content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/SSVM/SSVM_new(onehot)_tgan-org+balAR-11616.zip


/usr/local/lib/python3.7/dist-packages/pysurvival/utils/__init__.py:132: FutureWarning: 'pyarrow.serialize' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  model.save(path_file)


# tgan-org+balAR-16616

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from pysurvival.models.svm import LinearSVMModel
from pysurvival.models.simulations import SimulationModel
from pysurvival.utils.metrics import concordance_index
from sklearn.model_selection import train_test_split
from scipy.stats.stats import pearsonr

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/original+TGAN-balAR/origin+tganbalAR-16616.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [ ]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
data_train = new_train_df
data_test = new_test_df

In [ ]:
sim = SimulationModel( survival_distribution = 'exponential',
                       risk_type = 'linear',
                       censored_parameter = 1,
                       alpha = 3)

In [ ]:
features = [f for f in data_train.columns if f not in ['a_year_time', 'a_year_event']]

time_column = 'a_year_time'
event_column = 'a_year_event'

In [ ]:
X_train, X_test = data_train[features], data_test[features]
T_train, T_test = data_train['a_year_time'].values, data_test['a_year_time'].values
E_train, E_test = data_train['a_year_event'].values, data_test['a_year_event'].values

In [ ]:
from sklearn.preprocessing import StandardScaler

X_scaler = StandardScaler().fit(X_train)
X_train = X_scaler.transform(X_train)
X_test = X_scaler.transform(X_test)

In [ ]:
svm_model = LinearSVMModel()
svm_model.fit(X_train, T_train, E_train, init_method='he_uniform',
    with_bias = True, lr = 0.5,  tol = 1e-3,  l2_reg = 1e-3)

Performing Newton-Raphson optimization: 
 * Iteration #1 - Loss = 1648021.517 - ||grad||^2 = 2520291.73682 - ||diff_w|| = 19.46804
 * Iteration #2 - Loss = 432266.052 - ||grad||^2 = 1330091.39529 - ||diff_w|| = 8.72042
 * Iteration #3 - Loss = 143510.813 - ||grad||^2 = 794399.05786 - ||diff_w|| = 4.68442
 * Iteration #4 - Loss = 54452.489 - ||grad||^2 = 464443.56080 - ||diff_w|| = 2.20368
 * Iteration #5 - Loss = 29347.522 - ||grad||^2 = 192484.45483 - ||diff_w|| = 0.68335
 * Iteration #6 - Loss = 24668.983 - ||grad||^2 = 61207.57697 - ||diff_w|| = 0.20612
 * Iteration #7 - Loss = 24301.073 - ||grad||^2 = 25349.16364 - ||diff_w|| = 0.08876
 * Iteration #8 - Loss = 24289.228 - ||grad||^2 = 10331.80786 - ||diff_w|| = 0.03976
 * Iteration #9 - Loss = 24287.075 - ||grad||^2 = 4284.04201 - ||diff_w|| = 0.01868
 * Iteration #10 - Loss = 24286.096 - ||grad||^2 = 1861.84311 - ||diff_w|| = 0.00915
 * Iteration #11 - Loss = 24285.561 - ||grad||^2 = 852.71401 - ||diff_w|| = 0.00465
 * Iteration #

LinearSVMModel

In [ ]:
c_index = concordance_index(svm_model, X_test, T_test, E_test) #0.93
print('C-index: {:f}'.format(c_index))

C-index: 0.833274


In [ ]:
from pysurvival.utils import save_model

In [ ]:
save_model(svm_model,'/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/SSVM/SSVM_new(onehot)_tgan-org+balAR-16616.zip')

Saving the model to disk as /content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/SSVM/SSVM_new(onehot)_tgan-org+balAR-16616.zip


/usr/local/lib/python3.7/dist-packages/pysurvival/utils/__init__.py:132: FutureWarning: 'pyarrow.serialize' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  model.save(path_file)


# tgan-org+balAR-21616

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from pysurvival.models.svm import LinearSVMModel
from pysurvival.models.simulations import SimulationModel
from pysurvival.utils.metrics import concordance_index
from sklearn.model_selection import train_test_split
from scipy.stats.stats import pearsonr

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/original+TGAN-balAR/origin+tganbalAR-21616.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [ ]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
data_train = new_train_df
data_test = new_test_df

In [ ]:
sim = SimulationModel( survival_distribution = 'exponential',
                       risk_type = 'linear',
                       censored_parameter = 1,
                       alpha = 3)

In [ ]:
features = [f for f in data_train.columns if f not in ['a_year_time', 'a_year_event']]

time_column = 'a_year_time'
event_column = 'a_year_event'

In [ ]:
X_train, X_test = data_train[features], data_test[features]
T_train, T_test = data_train['a_year_time'].values, data_test['a_year_time'].values
E_train, E_test = data_train['a_year_event'].values, data_test['a_year_event'].values

In [ ]:
from sklearn.preprocessing import StandardScaler

X_scaler = StandardScaler().fit(X_train)
X_train = X_scaler.transform(X_train)
X_test = X_scaler.transform(X_test)

In [ ]:
svm_model = LinearSVMModel()
svm_model.fit(X_train, T_train, E_train, init_method='he_uniform',
    with_bias = True, lr = 0.5,  tol = 1e-3,  l2_reg = 1e-3)

Performing Newton-Raphson optimization: 
 * Iteration #1 - Loss = 9169573.246 - ||grad||^2 = 13034647.67127 - ||diff_w|| = 19.97380
 * Iteration #2 - Loss = 2315265.797 - ||grad||^2 = 6599596.53930 - ||diff_w|| = 8.97061
 * Iteration #3 - Loss = 709910.142 - ||grad||^2 = 3691645.42735 - ||diff_w|| = 4.89068
 * Iteration #4 - Loss = 221111.482 - ||grad||^2 = 2052123.53745 - ||diff_w|| = 2.56227
 * Iteration #5 - Loss = 78898.780 - ||grad||^2 = 1084619.60667 - ||diff_w|| = 1.18598
 * Iteration #6 - Loss = 45125.661 - ||grad||^2 = 539091.78311 - ||diff_w|| = 0.57701
 * Iteration #7 - Loss = 41060.052 - ||grad||^2 = 200326.12882 - ||diff_w|| = 0.30773
 * Iteration #8 - Loss = 41342.942 - ||grad||^2 = 63219.35458 - ||diff_w|| = 0.14225
 * Iteration #9 - Loss = 41339.533 - ||grad||^2 = 20265.05506 - ||diff_w|| = 0.06114
 * Iteration #10 - Loss = 41345.787 - ||grad||^2 = 6449.27568 - ||diff_w|| = 0.02660
 * Iteration #11 - Loss = 41344.564 - ||grad||^2 = 2202.92408 - ||diff_w|| = 0.01236
 * I

LinearSVMModel

In [ ]:
c_index = concordance_index(svm_model, X_test, T_test, E_test) #0.93
print('C-index: {:f}'.format(c_index))

C-index: 0.831799


In [ ]:
from pysurvival.utils import save_model

In [ ]:
save_model(svm_model,'/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/SSVM/SSVM_new(onehot)_tgan-org+balAR-21616.zip')

Saving the model to disk as /content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/SSVM/SSVM_new(onehot)_tgan-org+balAR-21616.zip


/usr/local/lib/python3.7/dist-packages/pysurvival/utils/__init__.py:132: FutureWarning: 'pyarrow.serialize' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  model.save(path_file)


# tgan-org+balAR-26616

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from pysurvival.models.svm import LinearSVMModel
from pysurvival.models.simulations import SimulationModel
from pysurvival.utils.metrics import concordance_index
from sklearn.model_selection import train_test_split
from scipy.stats.stats import pearsonr

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/original+TGAN-balAR/origin+tganbalAR-26616.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [ ]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
data_train = new_train_df
data_test = new_test_df

In [ ]:
sim = SimulationModel( survival_distribution = 'exponential',
                       risk_type = 'linear',
                       censored_parameter = 1,
                       alpha = 3)

In [ ]:
features = [f for f in data_train.columns if f not in ['a_year_time', 'a_year_event']]

time_column = 'a_year_time'
event_column = 'a_year_event'

In [ ]:
X_train, X_test = data_train[features], data_test[features]
T_train, T_test = data_train['a_year_time'].values, data_test['a_year_time'].values
E_train, E_test = data_train['a_year_event'].values, data_test['a_year_event'].values

In [ ]:
from sklearn.preprocessing import StandardScaler

X_scaler = StandardScaler().fit(X_train)
X_train = X_scaler.transform(X_train)
X_test = X_scaler.transform(X_test)

In [ ]:
svm_model = LinearSVMModel()
svm_model.fit(X_train, T_train, E_train, init_method='he_uniform',
    with_bias = True, lr = 0.5,  tol = 1e-3,  l2_reg = 1e-3)

Performing Newton-Raphson optimization: 
 * Iteration #1 - Loss = 2806989.674 - ||grad||^2 = 3874115.26573 - ||diff_w|| = 16.68777
 * Iteration #2 - Loss = 737799.016 - ||grad||^2 = 2043593.03166 - ||diff_w|| = 7.44599
 * Iteration #3 - Loss = 248512.918 - ||grad||^2 = 1204766.83486 - ||diff_w|| = 3.92354
 * Iteration #4 - Loss = 103634.546 - ||grad||^2 = 646437.02418 - ||diff_w|| = 1.79045
 * Iteration #5 - Loss = 68720.852 - ||grad||^2 = 194812.92683 - ||diff_w|| = 0.69112
 * Iteration #6 - Loss = 66045.181 - ||grad||^2 = 106819.76029 - ||diff_w|| = 0.28206
 * Iteration #7 - Loss = 65156.476 - ||grad||^2 = 41862.71124 - ||diff_w|| = 0.13370
 * Iteration #8 - Loss = 65055.250 - ||grad||^2 = 16748.07956 - ||diff_w|| = 0.06659
 * Iteration #9 - Loss = 65035.038 - ||grad||^2 = 6593.38341 - ||diff_w|| = 0.03365
 * Iteration #10 - Loss = 65043.074 - ||grad||^2 = 2250.34096 - ||diff_w|| = 0.01675
 * Iteration #11 - Loss = 65047.356 - ||grad||^2 = 880.45501 - ||diff_w|| = 0.00851
 * Iteratio

LinearSVMModel

In [ ]:
c_index = concordance_index(svm_model, X_test, T_test, E_test) #0.93
print('C-index: {:f}'.format(c_index))

C-index: 0.830786


In [ ]:
from pysurvival.utils import save_model

In [ ]:
save_model(svm_model,'/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/SSVM/SSVM_new(onehot)_tgan-org+balAR-26616.zip')

Saving the model to disk as /content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/SSVM/SSVM_new(onehot)_tgan-org+balAR-26616.zip


/usr/local/lib/python3.7/dist-packages/pysurvival/utils/__init__.py:132: FutureWarning: 'pyarrow.serialize' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  model.save(path_file)


# tgan-org+balAR-31616

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from pysurvival.models.svm import LinearSVMModel
from pysurvival.models.simulations import SimulationModel
from pysurvival.utils.metrics import concordance_index
from sklearn.model_selection import train_test_split
from scipy.stats.stats import pearsonr

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/original+TGAN-balAR/origin+tganbalAR-31616.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [ ]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
data_train = new_train_df
data_test = new_test_df

In [ ]:
sim = SimulationModel( survival_distribution = 'exponential',
                       risk_type = 'linear',
                       censored_parameter = 1,
                       alpha = 3)

In [ ]:
features = [f for f in data_train.columns if f not in ['a_year_time', 'a_year_event']]

time_column = 'a_year_time'
event_column = 'a_year_event'

In [ ]:
X_train, X_test = data_train[features], data_test[features]
T_train, T_test = data_train['a_year_time'].values, data_test['a_year_time'].values
E_train, E_test = data_train['a_year_event'].values, data_test['a_year_event'].values

In [ ]:
from sklearn.preprocessing import StandardScaler

X_scaler = StandardScaler().fit(X_train)
X_train = X_scaler.transform(X_train)
X_test = X_scaler.transform(X_test)

In [ ]:
svm_model = LinearSVMModel()
svm_model.fit(X_train, T_train, E_train, init_method='he_uniform',
    with_bias = True, lr = 0.5,  tol = 1e-3,  l2_reg = 1e-3)

Performing Newton-Raphson optimization: 
 * Iteration #1 - Loss = 19789841.925 - ||grad||^2 = 34421272.16309 - ||diff_w|| = 16.99451
 * Iteration #2 - Loss = 4997071.637 - ||grad||^2 = 17437284.69884 - ||diff_w|| = 7.62843
 * Iteration #3 - Loss = 1533113.339 - ||grad||^2 = 9758411.57806 - ||diff_w|| = 4.14943
 * Iteration #4 - Loss = 478787.084 - ||grad||^2 = 5425817.11825 - ||diff_w|| = 2.15801
 * Iteration #5 - Loss = 171669.233 - ||grad||^2 = 2875599.64962 - ||diff_w|| = 1.00215
 * Iteration #6 - Loss = 97671.306 - ||grad||^2 = 1428967.33866 - ||diff_w|| = 0.50531
 * Iteration #7 - Loss = 88442.171 - ||grad||^2 = 505982.06905 - ||diff_w|| = 0.26699
 * Iteration #8 - Loss = 88241.669 - ||grad||^2 = 150329.12548 - ||diff_w|| = 0.11967
 * Iteration #9 - Loss = 88001.085 - ||grad||^2 = 52624.89560 - ||diff_w|| = 0.05041
 * Iteration #10 - Loss = 87975.592 - ||grad||^2 = 18264.66795 - ||diff_w|| = 0.02162
 * Iteration #11 - Loss = 87981.112 - ||grad||^2 = 5559.59295 - ||diff_w|| = 0.008

LinearSVMModel

In [ ]:
c_index = concordance_index(svm_model, X_test, T_test, E_test) #0.93
print('C-index: {:f}'.format(c_index))

C-index: 0.831612


In [ ]:
from pysurvival.utils import save_model

In [ ]:
save_model(svm_model,'/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/SSVM/SSVM_new(onehot)_tgan-org+balAR-31616.zip')

Saving the model to disk as /content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/SSVM/SSVM_new(onehot)_tgan-org+balAR-31616.zip


/usr/local/lib/python3.7/dist-packages/pysurvival/utils/__init__.py:132: FutureWarning: 'pyarrow.serialize' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  model.save(path_file)


# tgan-org+balAR-36616

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from pysurvival.models.svm import LinearSVMModel
from pysurvival.models.simulations import SimulationModel
from pysurvival.utils.metrics import concordance_index
from sklearn.model_selection import train_test_split
from scipy.stats.stats import pearsonr

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/original+TGAN-balAR/origin+tganbalAR-36616.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [ ]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
data_train = new_train_df
data_test = new_test_df

In [ ]:
sim = SimulationModel( survival_distribution = 'exponential',
                       risk_type = 'linear',
                       censored_parameter = 1,
                       alpha = 3)

In [ ]:
features = [f for f in data_train.columns if f not in ['a_year_time', 'a_year_event']]

time_column = 'a_year_time'
event_column = 'a_year_event'

In [ ]:
X_train, X_test = data_train[features], data_test[features]
T_train, T_test = data_train['a_year_time'].values, data_test['a_year_time'].values
E_train, E_test = data_train['a_year_event'].values, data_test['a_year_event'].values

In [ ]:
from sklearn.preprocessing import StandardScaler

X_scaler = StandardScaler().fit(X_train)
X_train = X_scaler.transform(X_train)
X_test = X_scaler.transform(X_test)

In [ ]:
svm_model = LinearSVMModel()
svm_model.fit(X_train, T_train, E_train, init_method='he_uniform',
    with_bias = True, lr = 0.5,  tol = 1e-3,  l2_reg = 1e-3)

Performing Newton-Raphson optimization: 
 * Iteration #1 - Loss = 14506286.711 - ||grad||^2 = 24804731.97273 - ||diff_w|| = 15.77542
 * Iteration #2 - Loss = 3700452.605 - ||grad||^2 = 12611723.21722 - ||diff_w|| = 7.07872
 * Iteration #3 - Loss = 1159824.507 - ||grad||^2 = 7090180.54215 - ||diff_w|| = 3.84364
 * Iteration #4 - Loss = 385141.252 - ||grad||^2 = 3930534.25187 - ||diff_w|| = 1.97274
 * Iteration #5 - Loss = 163952.860 - ||grad||^2 = 1906931.06079 - ||diff_w|| = 0.82030
 * Iteration #6 - Loss = 124266.154 - ||grad||^2 = 795386.28097 - ||diff_w|| = 0.34784
 * Iteration #7 - Loss = 121360.458 - ||grad||^2 = 284409.36486 - ||diff_w|| = 0.16517
 * Iteration #8 - Loss = 121132.494 - ||grad||^2 = 95115.06451 - ||diff_w|| = 0.07421
 * Iteration #9 - Loss = 121131.119 - ||grad||^2 = 33138.63266 - ||diff_w|| = 0.03498
 * Iteration #10 - Loss = 121123.022 - ||grad||^2 = 12180.34713 - ||diff_w|| = 0.01724
 * Iteration #11 - Loss = 121118.763 - ||grad||^2 = 4841.98639 - ||diff_w|| = 0

LinearSVMModel

In [ ]:
c_index = concordance_index(svm_model, X_test, T_test, E_test) #0.93
print('C-index: {:f}'.format(c_index))

C-index: 0.830315


In [ ]:
from pysurvival.utils import save_model

In [ ]:
save_model(svm_model,'/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/SSVM/SSVM_new(onehot)_tgan-org+balAR-36616.zip')

Saving the model to disk as /content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/SSVM/SSVM_new(onehot)_tgan-org+balAR-36616.zip


/usr/local/lib/python3.7/dist-packages/pysurvival/utils/__init__.py:132: FutureWarning: 'pyarrow.serialize' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  model.save(path_file)


# tgan-org+balAR-41616

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from pysurvival.models.svm import LinearSVMModel
from pysurvival.models.simulations import SimulationModel
from pysurvival.utils.metrics import concordance_index
from sklearn.model_selection import train_test_split
from scipy.stats.stats import pearsonr

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/original+TGAN-balAR/origin+tganbalAR-41616.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [ ]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
data_train = new_train_df
data_test = new_test_df

In [ ]:
sim = SimulationModel( survival_distribution = 'exponential',
                       risk_type = 'linear',
                       censored_parameter = 1,
                       alpha = 3)

In [ ]:
features = [f for f in data_train.columns if f not in ['a_year_time', 'a_year_event']]

time_column = 'a_year_time'
event_column = 'a_year_event'

In [ ]:
X_train, X_test = data_train[features], data_test[features]
T_train, T_test = data_train['a_year_time'].values, data_test['a_year_time'].values
E_train, E_test = data_train['a_year_event'].values, data_test['a_year_event'].values

In [ ]:
from sklearn.preprocessing import StandardScaler

X_scaler = StandardScaler().fit(X_train)
X_train = X_scaler.transform(X_train)
X_test = X_scaler.transform(X_test)

In [ ]:
svm_model = LinearSVMModel()
svm_model.fit(X_train, T_train, E_train, init_method='he_uniform',
    with_bias = True, lr = 0.5,  tol = 1e-3,  l2_reg = 1e-3)

Performing Newton-Raphson optimization: 
 * Iteration #1 - Loss = 16356825.265 - ||grad||^2 = 27451213.53346 - ||diff_w|| = 16.81271
 * Iteration #2 - Loss = 4171842.398 - ||grad||^2 = 14004819.80317 - ||diff_w|| = 7.53866
 * Iteration #3 - Loss = 1309101.502 - ||grad||^2 = 7895021.84308 - ||diff_w|| = 4.07835
 * Iteration #4 - Loss = 440013.089 - ||grad||^2 = 4311780.44203 - ||diff_w|| = 2.04676
 * Iteration #5 - Loss = 205914.157 - ||grad||^2 = 2056146.00482 - ||diff_w|| = 0.76507
 * Iteration #6 - Loss = 166069.383 - ||grad||^2 = 926166.75215 - ||diff_w|| = 0.35218
 * Iteration #7 - Loss = 162571.203 - ||grad||^2 = 333223.82461 - ||diff_w|| = 0.14486
 * Iteration #8 - Loss = 162740.366 - ||grad||^2 = 107368.76843 - ||diff_w|| = 0.05608
 * Iteration #9 - Loss = 162766.014 - ||grad||^2 = 35109.77829 - ||diff_w|| = 0.02276
 * Iteration #10 - Loss = 162770.490 - ||grad||^2 = 12145.92484 - ||diff_w|| = 0.00974
 * Iteration #11 - Loss = 162775.575 - ||grad||^2 = 4473.36647 - ||diff_w|| = 

LinearSVMModel

In [ ]:
c_index = concordance_index(svm_model, X_test, T_test, E_test) #0.93
print('C-index: {:f}'.format(c_index))

C-index: 0.830892


In [ ]:
from pysurvival.utils import save_model

In [ ]:
save_model(svm_model,'/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/SSVM/SSVM_new(onehot)_tgan-org+balAR-41616.zip')

Saving the model to disk as /content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/SSVM/SSVM_new(onehot)_tgan-org+balAR-41616.zip


/usr/local/lib/python3.7/dist-packages/pysurvival/utils/__init__.py:132: FutureWarning: 'pyarrow.serialize' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  model.save(path_file)


# tgan-org+balAR-46616

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from pysurvival.models.svm import LinearSVMModel
from pysurvival.models.simulations import SimulationModel
from pysurvival.utils.metrics import concordance_index
from sklearn.model_selection import train_test_split
from scipy.stats.stats import pearsonr

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/original+TGAN-balAR/origin+tganbalAR-46616.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [ ]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
data_train = new_train_df
data_test = new_test_df

In [ ]:
sim = SimulationModel( survival_distribution = 'exponential',
                       risk_type = 'linear',
                       censored_parameter = 1,
                       alpha = 3)

In [ ]:
features = [f for f in data_train.columns if f not in ['a_year_time', 'a_year_event']]

time_column = 'a_year_time'
event_column = 'a_year_event'

In [ ]:
X_train, X_test = data_train[features], data_test[features]
T_train, T_test = data_train['a_year_time'].values, data_test['a_year_time'].values
E_train, E_test = data_train['a_year_event'].values, data_test['a_year_event'].values

In [ ]:
from sklearn.preprocessing import StandardScaler

X_scaler = StandardScaler().fit(X_train)
X_train = X_scaler.transform(X_train)
X_test = X_scaler.transform(X_test)

In [ ]:
svm_model = LinearSVMModel()
svm_model.fit(X_train, T_train, E_train, init_method='he_uniform',
    with_bias = True, lr = 0.5,  tol = 1e-3,  l2_reg = 1e-3)

Performing Newton-Raphson optimization: 
 * Iteration #1 - Loss = 34710766.835 - ||grad||^2 = 48843617.20646 - ||diff_w|| = 17.59845
 * Iteration #2 - Loss = 8790343.757 - ||grad||^2 = 24688483.60264 - ||diff_w|| = 7.90751
 * Iteration #3 - Loss = 2718163.441 - ||grad||^2 = 13761145.30731 - ||diff_w|| = 4.31623
 * Iteration #4 - Loss = 867518.812 - ||grad||^2 = 7569518.92302 - ||diff_w|| = 2.26759
 * Iteration #5 - Loss = 330270.006 - ||grad||^2 = 3763233.97894 - ||diff_w|| = 0.98907
 * Iteration #6 - Loss = 218247.486 - ||grad||^2 = 1731139.30715 - ||diff_w|| = 0.40929
 * Iteration #7 - Loss = 204271.874 - ||grad||^2 = 677772.63246 - ||diff_w|| = 0.17694
 * Iteration #8 - Loss = 203034.827 - ||grad||^2 = 242334.76457 - ||diff_w|| = 0.07034
 * Iteration #9 - Loss = 202711.911 - ||grad||^2 = 85811.79524 - ||diff_w|| = 0.02807
 * Iteration #10 - Loss = 202727.383 - ||grad||^2 = 28534.93146 - ||diff_w|| = 0.01012
 * Iteration #11 - Loss = 202748.122 - ||grad||^2 = 8719.03260 - ||diff_w|| 

LinearSVMModel

In [ ]:
c_index = concordance_index(svm_model, X_test, T_test, E_test) #0.93
print('C-index: {:f}'.format(c_index))

C-index: 0.830688


In [ ]:
from pysurvival.utils import save_model

In [ ]:
save_model(svm_model,'/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/SSVM/SSVM_new(onehot)_tgan-org+balAR-46616.zip')

Saving the model to disk as /content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/SSVM/SSVM_new(onehot)_tgan-org+balAR-46616.zip


/usr/local/lib/python3.7/dist-packages/pysurvival/utils/__init__.py:132: FutureWarning: 'pyarrow.serialize' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  model.save(path_file)


# tgan-org+balAR-51616

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from pysurvival.models.svm import LinearSVMModel
from pysurvival.models.simulations import SimulationModel
from pysurvival.utils.metrics import concordance_index
from sklearn.model_selection import train_test_split
from scipy.stats.stats import pearsonr

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/original+TGAN-balAR/origin+tganbalAR-51616.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [ ]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
data_train = new_train_df
data_test = new_test_df

In [ ]:
sim = SimulationModel( survival_distribution = 'exponential',
                       risk_type = 'linear',
                       censored_parameter = 1,
                       alpha = 3)

In [ ]:
features = [f for f in data_train.columns if f not in ['a_year_time', 'a_year_event']]

time_column = 'a_year_time'
event_column = 'a_year_event'

In [ ]:
X_train, X_test = data_train[features], data_test[features]
T_train, T_test = data_train['a_year_time'].values, data_test['a_year_time'].values
E_train, E_test = data_train['a_year_event'].values, data_test['a_year_event'].values

In [ ]:
from sklearn.preprocessing import StandardScaler

X_scaler = StandardScaler().fit(X_train)
X_train = X_scaler.transform(X_train)
X_test = X_scaler.transform(X_test)

In [ ]:
svm_model = LinearSVMModel()
svm_model.fit(X_train, T_train, E_train, init_method='he_uniform',
    with_bias = True, lr = 0.5,  tol = 1e-3,  l2_reg = 1e-3)

Performing Newton-Raphson optimization: 
 * Iteration #1 - Loss = 23513975.338 - ||grad||^2 = 39155053.49174 - ||diff_w|| = 20.15564
 * Iteration #2 - Loss = 5988631.991 - ||grad||^2 = 19889842.47950 - ||diff_w|| = 9.03482
 * Iteration #3 - Loss = 1874248.850 - ||grad||^2 = 11112454.44268 - ||diff_w|| = 4.87276
 * Iteration #4 - Loss = 633194.259 - ||grad||^2 = 5895222.99888 - ||diff_w|| = 2.39138
 * Iteration #5 - Loss = 316552.110 - ||grad||^2 = 2853557.91232 - ||diff_w|| = 1.03746
 * Iteration #6 - Loss = 263547.536 - ||grad||^2 = 1166043.62299 - ||diff_w|| = 0.48122
 * Iteration #7 - Loss = 259039.883 - ||grad||^2 = 431743.11857 - ||diff_w|| = 0.21948
 * Iteration #8 - Loss = 258329.588 - ||grad||^2 = 154317.10767 - ||diff_w|| = 0.09687
 * Iteration #9 - Loss = 258305.825 - ||grad||^2 = 52017.62772 - ||diff_w|| = 0.04327
 * Iteration #10 - Loss = 258338.793 - ||grad||^2 = 17508.95930 - ||diff_w|| = 0.02070
 * Iteration #11 - Loss = 258351.465 - ||grad||^2 = 6207.30067 - ||diff_w|| 

LinearSVMModel

In [ ]:
c_index = concordance_index(svm_model, X_test, T_test, E_test) #0.93
print('C-index: {:f}'.format(c_index))

C-index: 0.830279


In [ ]:
from pysurvival.utils import save_model

In [ ]:
save_model(svm_model,'/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/SSVM/SSVM_new(onehot)_tgan-org+balAR-51616.zip')

Saving the model to disk as /content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/SSVM/SSVM_new(onehot)_tgan-org+balAR-51616.zip


/usr/local/lib/python3.7/dist-packages/pysurvival/utils/__init__.py:132: FutureWarning: 'pyarrow.serialize' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  model.save(path_file)


# orig+tgan-aug-6616

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from pysurvival.models.svm import LinearSVMModel
from pysurvival.models.simulations import SimulationModel
from pysurvival.utils.metrics import concordance_index
from sklearn.model_selection import train_test_split
from scipy.stats.stats import pearsonr

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/Original+TGAN-aug/orig-TGANaug-6616.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [ ]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
data_train = new_train_df
data_test = new_test_df

In [ ]:
sim = SimulationModel( survival_distribution = 'exponential',
                       risk_type = 'linear',
                       censored_parameter = 1,
                       alpha = 3)

In [ ]:
features = [f for f in data_train.columns if f not in ['a_year_time', 'a_year_event']]

time_column = 'a_year_time'
event_column = 'a_year_event'

In [ ]:
X_train, X_test = data_train[features], data_test[features]
T_train, T_test = data_train['a_year_time'].values, data_test['a_year_time'].values
E_train, E_test = data_train['a_year_event'].values, data_test['a_year_event'].values

In [ ]:
from sklearn.preprocessing import StandardScaler

X_scaler = StandardScaler().fit(X_train)
X_train = X_scaler.transform(X_train)
X_test = X_scaler.transform(X_test)

In [ ]:
svm_model = LinearSVMModel()
svm_model.fit(X_train, T_train, E_train, init_method='he_uniform',
    with_bias = True, lr = 0.5,  tol = 1e-3,  l2_reg = 1e-3)

Performing Newton-Raphson optimization: 
 * Iteration #1 - Loss = 203490.267 - ||grad||^2 = 253112.10863 - ||diff_w|| = 16.08824
 * Iteration #2 - Loss = 52513.804 - ||grad||^2 = 129946.50532 - ||diff_w|| = 7.21471
 * Iteration #3 - Loss = 16870.611 - ||grad||^2 = 74951.85062 - ||diff_w|| = 3.90716
 * Iteration #4 - Loss = 5900.117 - ||grad||^2 = 43744.78429 - ||diff_w|| = 1.96547
 * Iteration #5 - Loss = 2693.929 - ||grad||^2 = 20283.05800 - ||diff_w|| = 0.74869
 * Iteration #6 - Loss = 2051.945 - ||grad||^2 = 8111.82582 - ||diff_w|| = 0.29658
 * Iteration #7 - Loss = 1995.352 - ||grad||^2 = 3335.71833 - ||diff_w|| = 0.13771
 * Iteration #8 - Loss = 1995.699 - ||grad||^2 = 1249.90275 - ||diff_w|| = 0.06297
 * Iteration #9 - Loss = 1996.886 - ||grad||^2 = 480.36079 - ||diff_w|| = 0.03022
 * Iteration #10 - Loss = 1997.304 - ||grad||^2 = 198.05305 - ||diff_w|| = 0.01485
 * Iteration #11 - Loss = 1997.408 - ||grad||^2 = 90.17372 - ||diff_w|| = 0.00763
 * Iteration #12 - Loss = 1997.440 -

LinearSVMModel

In [ ]:
c_index = concordance_index(svm_model, X_test, T_test, E_test) #0.93
print('C-index: {:f}'.format(c_index))

C-index: 0.836704


In [ ]:
from pysurvival.utils import save_model

In [ ]:
save_model(svm_model,'/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/SSVM/SSVM_new(onehot)_tgan-org+imbalnonar-6616.zip')

Saving the model to disk as /content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/SSVM/SSVM_new(onehot)_tgan-org+imbalnonar-6616.zip


/usr/local/lib/python3.7/dist-packages/pysurvival/utils/__init__.py:132: FutureWarning: 'pyarrow.serialize' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  model.save(path_file)


# orig+tgan-aug-11616

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from pysurvival.models.svm import LinearSVMModel
from pysurvival.models.simulations import SimulationModel
from pysurvival.utils.metrics import concordance_index
from sklearn.model_selection import train_test_split
from scipy.stats.stats import pearsonr

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/Original+TGAN-aug/orig-TGANaug-11616.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [ ]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
data_train = new_train_df
data_test = new_test_df

In [ ]:
sim = SimulationModel( survival_distribution = 'exponential',
                       risk_type = 'linear',
                       censored_parameter = 1,
                       alpha = 3)

In [ ]:
features = [f for f in data_train.columns if f not in ['a_year_time', 'a_year_event']]

time_column = 'a_year_time'
event_column = 'a_year_event'

In [ ]:
X_train, X_test = data_train[features], data_test[features]
T_train, T_test = data_train['a_year_time'].values, data_test['a_year_time'].values
E_train, E_test = data_train['a_year_event'].values, data_test['a_year_event'].values

In [ ]:
from sklearn.preprocessing import StandardScaler

X_scaler = StandardScaler().fit(X_train)
X_train = X_scaler.transform(X_train)
X_test = X_scaler.transform(X_test)

In [ ]:
svm_model = LinearSVMModel()
svm_model.fit(X_train, T_train, E_train, init_method='he_uniform',
    with_bias = True, lr = 0.5,  tol = 1e-3,  l2_reg = 1e-3)

Performing Newton-Raphson optimization: 
 * Iteration #1 - Loss = 704144.425 - ||grad||^2 = 1450305.37871 - ||diff_w|| = 17.29851
 * Iteration #2 - Loss = 178534.536 - ||grad||^2 = 739786.59858 - ||diff_w|| = 7.75262
 * Iteration #3 - Loss = 55093.685 - ||grad||^2 = 416057.24638 - ||diff_w|| = 4.17929
 * Iteration #4 - Loss = 17900.788 - ||grad||^2 = 228085.50410 - ||diff_w|| = 2.08625
 * Iteration #5 - Loss = 8105.943 - ||grad||^2 = 115178.51023 - ||diff_w|| = 0.97240
 * Iteration #6 - Loss = 6201.352 - ||grad||^2 = 50414.42709 - ||diff_w|| = 0.49870
 * Iteration #7 - Loss = 6012.064 - ||grad||^2 = 16882.83031 - ||diff_w|| = 0.25461
 * Iteration #8 - Loss = 6034.850 - ||grad||^2 = 5612.92741 - ||diff_w|| = 0.12058
 * Iteration #9 - Loss = 6037.709 - ||grad||^2 = 2432.07038 - ||diff_w|| = 0.05741
 * Iteration #10 - Loss = 6036.447 - ||grad||^2 = 1193.74637 - ||diff_w|| = 0.02918
 * Iteration #11 - Loss = 6036.021 - ||grad||^2 = 606.16770 - ||diff_w|| = 0.01513
 * Iteration #12 - Loss =

LinearSVMModel

In [ ]:
c_index = concordance_index(svm_model, X_test, T_test, E_test) #0.93
print('C-index: {:f}'.format(c_index))

C-index: 0.833025


In [ ]:
from pysurvival.utils import save_model

In [ ]:
save_model(svm_model,'/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/SSVM/SSVM_new(onehot)_tgan-org+imbalnonar-11616.zip')

Saving the model to disk as /content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/SSVM/SSVM_new(onehot)_tgan-org+imbalnonar-11616.zip


/usr/local/lib/python3.7/dist-packages/pysurvival/utils/__init__.py:132: FutureWarning: 'pyarrow.serialize' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  model.save(path_file)


# orig+tgan-aug-16166

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from pysurvival.models.svm import LinearSVMModel
from pysurvival.models.simulations import SimulationModel
from pysurvival.utils.metrics import concordance_index
from sklearn.model_selection import train_test_split
from scipy.stats.stats import pearsonr

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/Original+TGAN-aug/orig-TGANaug-16166.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [ ]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
data_train = new_train_df
data_test = new_test_df

In [ ]:
sim = SimulationModel( survival_distribution = 'exponential',
                       risk_type = 'linear',
                       censored_parameter = 1,
                       alpha = 3)

In [ ]:
features = [f for f in data_train.columns if f not in ['a_year_time', 'a_year_event']]

time_column = 'a_year_time'
event_column = 'a_year_event'

In [ ]:
X_train, X_test = data_train[features], data_test[features]
T_train, T_test = data_train['a_year_time'].values, data_test['a_year_time'].values
E_train, E_test = data_train['a_year_event'].values, data_test['a_year_event'].values

In [ ]:
from sklearn.preprocessing import StandardScaler

X_scaler = StandardScaler().fit(X_train)
X_train = X_scaler.transform(X_train)
X_test = X_scaler.transform(X_test)

In [ ]:
svm_model = LinearSVMModel()
svm_model.fit(X_train, T_train, E_train, init_method='he_uniform',
    with_bias = True, lr = 0.5,  tol = 1e-3,  l2_reg = 1e-3)

Performing Newton-Raphson optimization: 
 * Iteration #1 - Loss = 733735.962 - ||grad||^2 = 874609.03463 - ||diff_w|| = 19.05403
 * Iteration #2 - Loss = 192389.902 - ||grad||^2 = 463493.59858 - ||diff_w|| = 8.52304
 * Iteration #3 - Loss = 64031.121 - ||grad||^2 = 280672.36129 - ||diff_w|| = 4.55430
 * Iteration #4 - Loss = 24698.105 - ||grad||^2 = 172643.10239 - ||diff_w|| = 2.11839
 * Iteration #5 - Loss = 13853.343 - ||grad||^2 = 71197.35839 - ||diff_w|| = 0.72252
 * Iteration #6 - Loss = 12005.294 - ||grad||^2 = 21421.86008 - ||diff_w|| = 0.29737
 * Iteration #7 - Loss = 11835.187 - ||grad||^2 = 9165.96154 - ||diff_w|| = 0.14320
 * Iteration #8 - Loss = 11824.487 - ||grad||^2 = 4233.94120 - ||diff_w|| = 0.07220
 * Iteration #9 - Loss = 11823.003 - ||grad||^2 = 2105.14529 - ||diff_w|| = 0.03719
 * Iteration #10 - Loss = 11822.556 - ||grad||^2 = 1086.68866 - ||diff_w|| = 0.01959
 * Iteration #11 - Loss = 11822.384 - ||grad||^2 = 571.75856 - ||diff_w|| = 0.01055
 * Iteration #12 - Lo

LinearSVMModel

In [ ]:
c_index = concordance_index(svm_model, X_test, T_test, E_test) #0.93
print('C-index: {:f}'.format(c_index))

C-index: 0.832634


In [ ]:
from pysurvival.utils import save_model

In [ ]:
save_model(svm_model,'/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/SSVM/SSVM_new(onehot)_tgan-org+imbalnonar-16166.zip')

Saving the model to disk as /content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/SSVM/SSVM_new(onehot)_tgan-org+imbalnonar-16166.zip


/usr/local/lib/python3.7/dist-packages/pysurvival/utils/__init__.py:132: FutureWarning: 'pyarrow.serialize' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  model.save(path_file)


# orig+tgan-aug-21616

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from pysurvival.models.svm import LinearSVMModel
from pysurvival.models.simulations import SimulationModel
from pysurvival.utils.metrics import concordance_index
from sklearn.model_selection import train_test_split
from scipy.stats.stats import pearsonr

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/Original+TGAN-aug/orig-TGANaug-21616.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [ ]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [ ]:
data_train = new_train_df
data_test = new_test_df

In [ ]:
sim = SimulationModel( survival_distribution = 'exponential',
                       risk_type = 'linear',
                       censored_parameter = 1,
                       alpha = 3)

In [ ]:
features = [f for f in data_train.columns if f not in ['a_year_time', 'a_year_event']]

time_column = 'a_year_time'
event_column = 'a_year_event'

In [ ]:
X_train, X_test = data_train[features], data_test[features]
T_train, T_test = data_train['a_year_time'].values, data_test['a_year_time'].values
E_train, E_test = data_train['a_year_event'].values, data_test['a_year_event'].values

In [ ]:
from sklearn.preprocessing import StandardScaler

X_scaler = StandardScaler().fit(X_train)
X_train = X_scaler.transform(X_train)
X_test = X_scaler.transform(X_test)

In [ ]:
svm_model = LinearSVMModel()
svm_model.fit(X_train, T_train, E_train, init_method='he_uniform',
    with_bias = True, lr = 0.5,  tol = 1e-3,  l2_reg = 1e-3)

Performing Newton-Raphson optimization: 
 * Iteration #1 - Loss = 1321147.832 - ||grad||^2 = 2397796.93459 - ||diff_w|| = 17.28983
 * Iteration #2 - Loss = 341179.952 - ||grad||^2 = 1225498.35268 - ||diff_w|| = 7.74826
 * Iteration #3 - Loss = 110377.819 - ||grad||^2 = 691004.98276 - ||diff_w|| = 4.15735
 * Iteration #4 - Loss = 40604.686 - ||grad||^2 = 366791.41837 - ||diff_w|| = 1.93078
 * Iteration #5 - Loss = 22757.238 - ||grad||^2 = 153094.52122 - ||diff_w|| = 0.62442
 * Iteration #6 - Loss = 20058.036 - ||grad||^2 = 69108.75158 - ||diff_w|| = 0.25452
 * Iteration #7 - Loss = 19774.622 - ||grad||^2 = 29532.02916 - ||diff_w|| = 0.11913
 * Iteration #8 - Loss = 19761.490 - ||grad||^2 = 12220.69139 - ||diff_w|| = 0.05684
 * Iteration #9 - Loss = 19759.280 - ||grad||^2 = 5484.26635 - ||diff_w|| = 0.02821
 * Iteration #10 - Loss = 19757.920 - ||grad||^2 = 2623.68521 - ||diff_w|| = 0.01473
 * Iteration #11 - Loss = 19757.397 - ||grad||^2 = 1312.49394 - ||diff_w|| = 0.00786
 * Iteration 

LinearSVMModel

In [ ]:
c_index = concordance_index(svm_model, X_test, T_test, E_test) #0.93
print('C-index: {:f}'.format(c_index))

C-index: 0.832625


In [ ]:
from pysurvival.utils import save_model

In [ ]:
save_model(svm_model,'/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/SSVM/SSVM_new(onehot)_tgan-org+imbalnonar-21616.zip')

Saving the model to disk as /content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/SSVM/SSVM_new(onehot)_tgan-org+imbalnonar-21616.zip


/usr/local/lib/python3.7/dist-packages/pysurvival/utils/__init__.py:132: FutureWarning: 'pyarrow.serialize' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  model.save(path_file)


# orig+tgan-aug-26616

In [2]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from pysurvival.models.svm import LinearSVMModel
from pysurvival.models.simulations import SimulationModel
from pysurvival.utils.metrics import concordance_index
from sklearn.model_selection import train_test_split
from scipy.stats.stats import pearsonr

In [3]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/Original+TGAN-aug/orig-TGANaug-26616.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [4]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [5]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [6]:
data_train = new_train_df
data_test = new_test_df

In [7]:
sim = SimulationModel( survival_distribution = 'exponential',
                       risk_type = 'linear',
                       censored_parameter = 1,
                       alpha = 3)

In [8]:
features = [f for f in data_train.columns if f not in ['a_year_time', 'a_year_event']]

time_column = 'a_year_time'
event_column = 'a_year_event'

In [9]:
X_train, X_test = data_train[features], data_test[features]
T_train, T_test = data_train['a_year_time'].values, data_test['a_year_time'].values
E_train, E_test = data_train['a_year_event'].values, data_test['a_year_event'].values

In [10]:
from sklearn.preprocessing import StandardScaler

X_scaler = StandardScaler().fit(X_train)
X_train = X_scaler.transform(X_train)
X_test = X_scaler.transform(X_test)

In [11]:
svm_model = LinearSVMModel()
svm_model.fit(X_train, T_train, E_train, init_method='he_uniform',
    with_bias = True, lr = 0.5,  tol = 1e-3,  l2_reg = 1e-3)

Performing Newton-Raphson optimization: 
 * Iteration #1 - Loss = 3449698.175 - ||grad||^2 = 5994436.20057 - ||diff_w|| = 15.85622
 * Iteration #2 - Loss = 875554.650 - ||grad||^2 = 3058129.77052 - ||diff_w|| = 7.11267
 * Iteration #3 - Loss = 271159.067 - ||grad||^2 = 1718999.17896 - ||diff_w|| = 3.85047
 * Iteration #4 - Loss = 88963.886 - ||grad||^2 = 939673.61012 - ||diff_w|| = 1.96270
 * Iteration #5 - Loss = 40422.500 - ||grad||^2 = 485183.60818 - ||diff_w|| = 0.95902
 * Iteration #6 - Loss = 30280.687 - ||grad||^2 = 229258.82186 - ||diff_w|| = 0.50062
 * Iteration #7 - Loss = 29148.860 - ||grad||^2 = 93201.13263 - ||diff_w|| = 0.24420
 * Iteration #8 - Loss = 29328.064 - ||grad||^2 = 33093.35967 - ||diff_w|| = 0.10483
 * Iteration #9 - Loss = 29391.734 - ||grad||^2 = 11864.77023 - ||diff_w|| = 0.04680
 * Iteration #10 - Loss = 29402.348 - ||grad||^2 = 4890.16957 - ||diff_w|| = 0.02306
 * Iteration #11 - Loss = 29404.352 - ||grad||^2 = 2131.38659 - ||diff_w|| = 0.01214
 * Iterati

LinearSVMModel

In [12]:
c_index = concordance_index(svm_model, X_test, T_test, E_test) #0.93
print('C-index: {:f}'.format(c_index))

C-index: 0.831772


In [13]:
from pysurvival.utils import save_model

In [14]:
save_model(svm_model,'/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/SSVM/SSVM_new(onehot)_tgan-org+imbalnonar-26616.zip')

Saving the model to disk as /content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/SSVM/SSVM_new(onehot)_tgan-org+imbalnonar-26616.zip


/usr/local/lib/python3.7/dist-packages/pysurvival/utils/__init__.py:132: FutureWarning: 'pyarrow.serialize' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  model.save(path_file)


# orig+tgan-aug-31616

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from pysurvival.models.svm import LinearSVMModel
from pysurvival.models.simulations import SimulationModel
from pysurvival.utils.metrics import concordance_index
from sklearn.model_selection import train_test_split
from scipy.stats.stats import pearsonr

In [2]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/Original+TGAN-aug/orig-TGANaug-31616.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [3]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [4]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [5]:
data_train = new_train_df
data_test = new_test_df

In [6]:
sim = SimulationModel( survival_distribution = 'exponential',
                       risk_type = 'linear',
                       censored_parameter = 1,
                       alpha = 3)

In [7]:
features = [f for f in data_train.columns if f not in ['a_year_time', 'a_year_event']]

time_column = 'a_year_time'
event_column = 'a_year_event'

In [8]:
X_train, X_test = data_train[features], data_test[features]
T_train, T_test = data_train['a_year_time'].values, data_test['a_year_time'].values
E_train, E_test = data_train['a_year_event'].values, data_test['a_year_event'].values

In [9]:
from sklearn.preprocessing import StandardScaler

X_scaler = StandardScaler().fit(X_train)
X_train = X_scaler.transform(X_train)
X_test = X_scaler.transform(X_test)

In [10]:
svm_model = LinearSVMModel()
svm_model.fit(X_train, T_train, E_train, init_method='he_uniform',
    with_bias = True, lr = 0.5,  tol = 1e-3,  l2_reg = 1e-3)

Performing Newton-Raphson optimization: 
 * Iteration #1 - Loss = 5595516.228 - ||grad||^2 = 7219684.39350 - ||diff_w|| = 18.36435
 * Iteration #2 - Loss = 1440413.458 - ||grad||^2 = 3720661.68984 - ||diff_w|| = 8.24042
 * Iteration #3 - Loss = 460004.013 - ||grad||^2 = 2159669.90794 - ||diff_w|| = 4.47607
 * Iteration #4 - Loss = 156523.985 - ||grad||^2 = 1278670.33249 - ||diff_w|| = 2.30719
 * Iteration #5 - Loss = 64433.127 - ||grad||^2 = 647903.02848 - ||diff_w|| = 0.94541
 * Iteration #6 - Loss = 43025.751 - ||grad||^2 = 211343.79531 - ||diff_w|| = 0.30925
 * Iteration #7 - Loss = 40613.676 - ||grad||^2 = 82586.48331 - ||diff_w|| = 0.12323
 * Iteration #8 - Loss = 40611.470 - ||grad||^2 = 30498.16986 - ||diff_w|| = 0.05230
 * Iteration #9 - Loss = 40642.020 - ||grad||^2 = 11562.41282 - ||diff_w|| = 0.02316
 * Iteration #10 - Loss = 40650.803 - ||grad||^2 = 4847.20146 - ||diff_w|| = 0.01137
 * Iteration #11 - Loss = 40653.434 - ||grad||^2 = 2176.07860 - ||diff_w|| = 0.00576
 * Iter

LinearSVMModel

In [11]:
c_index = concordance_index(svm_model, X_test, T_test, E_test) #0.93
print('C-index: {:f}'.format(c_index))

C-index: 0.832190


In [12]:
from pysurvival.utils import save_model

In [13]:
save_model(svm_model,'/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/SSVM/SSVM_new(onehot)_tgan-org+imbalnonar-31616.zip')

Saving the model to disk as /content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/SSVM/SSVM_new(onehot)_tgan-org+imbalnonar-31616.zip


/usr/local/lib/python3.7/dist-packages/pysurvival/utils/__init__.py:132: FutureWarning: 'pyarrow.serialize' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  model.save(path_file)


# orig+tgan-aug-36616

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from pysurvival.models.svm import LinearSVMModel
from pysurvival.models.simulations import SimulationModel
from pysurvival.utils.metrics import concordance_index
from sklearn.model_selection import train_test_split
from scipy.stats.stats import pearsonr

In [2]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/Original+TGAN-aug/orig-TGANaug-36616.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [3]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [4]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [5]:
data_train = new_train_df
data_test = new_test_df

In [6]:
sim = SimulationModel( survival_distribution = 'exponential',
                       risk_type = 'linear',
                       censored_parameter = 1,
                       alpha = 3)

In [7]:
features = [f for f in data_train.columns if f not in ['a_year_time', 'a_year_event']]

time_column = 'a_year_time'
event_column = 'a_year_event'

In [8]:
X_train, X_test = data_train[features], data_test[features]
T_train, T_test = data_train['a_year_time'].values, data_test['a_year_time'].values
E_train, E_test = data_train['a_year_event'].values, data_test['a_year_event'].values

In [9]:
from sklearn.preprocessing import StandardScaler

X_scaler = StandardScaler().fit(X_train)
X_train = X_scaler.transform(X_train)
X_test = X_scaler.transform(X_test)

In [10]:
svm_model = LinearSVMModel()
svm_model.fit(X_train, T_train, E_train, init_method='he_uniform',
    with_bias = True, lr = 0.5,  tol = 1e-3,  l2_reg = 1e-3)

Performing Newton-Raphson optimization: 
 * Iteration #1 - Loss = 5282882.784 - ||grad||^2 = 9505055.43098 - ||diff_w|| = 14.39728
 * Iteration #2 - Loss = 1341747.562 - ||grad||^2 = 4842934.19695 - ||diff_w|| = 6.45116
 * Iteration #3 - Loss = 417612.809 - ||grad||^2 = 2718853.35357 - ||diff_w|| = 3.46729
 * Iteration #4 - Loss = 140015.033 - ||grad||^2 = 1451349.42903 - ||diff_w|| = 1.66951
 * Iteration #5 - Loss = 69186.511 - ||grad||^2 = 686537.43885 - ||diff_w|| = 0.69851
 * Iteration #6 - Loss = 56598.129 - ||grad||^2 = 323069.28156 - ||diff_w|| = 0.32725
 * Iteration #7 - Loss = 55068.947 - ||grad||^2 = 134903.55994 - ||diff_w|| = 0.14049
 * Iteration #8 - Loss = 55129.120 - ||grad||^2 = 51872.82999 - ||diff_w|| = 0.05540
 * Iteration #9 - Loss = 55179.657 - ||grad||^2 = 20740.92624 - ||diff_w|| = 0.02422
 * Iteration #10 - Loss = 55187.177 - ||grad||^2 = 8752.62313 - ||diff_w|| = 0.01147
 * Iteration #11 - Loss = 55188.807 - ||grad||^2 = 3918.49295 - ||diff_w|| = 0.00565
 * Ite

LinearSVMModel

In [11]:
c_index = concordance_index(svm_model, X_test, T_test, E_test) #0.93
print('C-index: {:f}'.format(c_index))

C-index: 0.832821


In [12]:
from pysurvival.utils import save_model

In [13]:
save_model(svm_model,'/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/SSVM/SSVM_new(onehot)_tgan-org+imbalnonar-36616.zip')

Saving the model to disk as /content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/SSVM/SSVM_new(onehot)_tgan-org+imbalnonar-36616.zip


/usr/local/lib/python3.7/dist-packages/pysurvival/utils/__init__.py:132: FutureWarning: 'pyarrow.serialize' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  model.save(path_file)


# orig+tgan-aug-41616

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from pysurvival.models.svm import LinearSVMModel
from pysurvival.models.simulations import SimulationModel
from pysurvival.utils.metrics import concordance_index
from sklearn.model_selection import train_test_split
from scipy.stats.stats import pearsonr

In [2]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/Original+TGAN-aug/orig-TGANaug-41616.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [3]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [4]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [5]:
data_train = new_train_df
data_test = new_test_df

In [6]:
sim = SimulationModel( survival_distribution = 'exponential',
                       risk_type = 'linear',
                       censored_parameter = 1,
                       alpha = 3)

In [7]:
features = [f for f in data_train.columns if f not in ['a_year_time', 'a_year_event']]

time_column = 'a_year_time'
event_column = 'a_year_event'

In [8]:
X_train, X_test = data_train[features], data_test[features]
T_train, T_test = data_train['a_year_time'].values, data_test['a_year_time'].values
E_train, E_test = data_train['a_year_event'].values, data_test['a_year_event'].values

In [9]:
from sklearn.preprocessing import StandardScaler

X_scaler = StandardScaler().fit(X_train)
X_train = X_scaler.transform(X_train)
X_test = X_scaler.transform(X_test)

In [10]:
svm_model = LinearSVMModel()
svm_model.fit(X_train, T_train, E_train, init_method='he_uniform',
    with_bias = True, lr = 0.5,  tol = 1e-3,  l2_reg = 1e-3)

Performing Newton-Raphson optimization: 
 * Iteration #1 - Loss = 8405249.369 - ||grad||^2 = 16152578.61603 - ||diff_w|| = 14.62285
 * Iteration #2 - Loss = 2142794.589 - ||grad||^2 = 8207031.45745 - ||diff_w|| = 6.55328
 * Iteration #3 - Loss = 671152.154 - ||grad||^2 = 4598609.01085 - ||diff_w|| = 3.54283
 * Iteration #4 - Loss = 223331.250 - ||grad||^2 = 2512358.55358 - ||diff_w|| = 1.78431
 * Iteration #5 - Loss = 99369.693 - ||grad||^2 = 1280801.36078 - ||diff_w|| = 0.77960
 * Iteration #6 - Loss = 74709.966 - ||grad||^2 = 597434.91661 - ||diff_w|| = 0.38560
 * Iteration #7 - Loss = 72624.406 - ||grad||^2 = 208236.46323 - ||diff_w|| = 0.18087
 * Iteration #8 - Loss = 73045.267 - ||grad||^2 = 62479.84276 - ||diff_w|| = 0.07508
 * Iteration #9 - Loss = 73168.923 - ||grad||^2 = 20615.38668 - ||diff_w|| = 0.03130
 * Iteration #10 - Loss = 73193.930 - ||grad||^2 = 8272.44652 - ||diff_w|| = 0.01426
 * Iteration #11 - Loss = 73199.811 - ||grad||^2 = 3577.25790 - ||diff_w|| = 0.00733
 * I

LinearSVMModel

In [11]:
c_index = concordance_index(svm_model, X_test, T_test, E_test) #0.93
print('C-index: {:f}'.format(c_index))

C-index: 0.831746


In [12]:
from pysurvival.utils import save_model

In [13]:
save_model(svm_model,'/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/SSVM/SSVM_new(onehot)_tgan-org+imbalnonar-41616.zip')

Saving the model to disk as /content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/SSVM/SSVM_new(onehot)_tgan-org+imbalnonar-41616.zip


/usr/local/lib/python3.7/dist-packages/pysurvival/utils/__init__.py:132: FutureWarning: 'pyarrow.serialize' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  model.save(path_file)


# orig+tgan-aug-46616

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from pysurvival.models.svm import LinearSVMModel
from pysurvival.models.simulations import SimulationModel
from pysurvival.utils.metrics import concordance_index
from sklearn.model_selection import train_test_split
from scipy.stats.stats import pearsonr

In [2]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/Original+TGAN-aug/orig-TGANaug-46616.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [3]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [4]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [5]:
data_train = new_train_df
data_test = new_test_df

In [6]:
sim = SimulationModel( survival_distribution = 'exponential',
                       risk_type = 'linear',
                       censored_parameter = 1,
                       alpha = 3)

In [7]:
features = [f for f in data_train.columns if f not in ['a_year_time', 'a_year_event']]

time_column = 'a_year_time'
event_column = 'a_year_event'

In [8]:
X_train, X_test = data_train[features], data_test[features]
T_train, T_test = data_train['a_year_time'].values, data_test['a_year_time'].values
E_train, E_test = data_train['a_year_event'].values, data_test['a_year_event'].values

In [9]:
from sklearn.preprocessing import StandardScaler

X_scaler = StandardScaler().fit(X_train)
X_train = X_scaler.transform(X_train)
X_test = X_scaler.transform(X_test)

In [10]:
svm_model = LinearSVMModel()
svm_model.fit(X_train, T_train, E_train, init_method='he_uniform',
    with_bias = True, lr = 0.5,  tol = 1e-3,  l2_reg = 1e-3)

Performing Newton-Raphson optimization: 
 * Iteration #1 - Loss = 9402512.367 - ||grad||^2 = 23424228.11724 - ||diff_w|| = 10.63407
 * Iteration #2 - Loss = 2395303.016 - ||grad||^2 = 11969087.70567 - ||diff_w|| = 4.74872
 * Iteration #3 - Loss = 746906.862 - ||grad||^2 = 6734278.89914 - ||diff_w|| = 2.52633
 * Iteration #4 - Loss = 250006.195 - ||grad||^2 = 3666822.40956 - ||diff_w|| = 1.20144
 * Iteration #5 - Loss = 117680.747 - ||grad||^2 = 1921456.43595 - ||diff_w|| = 0.53960
 * Iteration #6 - Loss = 89054.932 - ||grad||^2 = 900482.66008 - ||diff_w|| = 0.28914
 * Iteration #7 - Loss = 88464.422 - ||grad||^2 = 265603.81430 - ||diff_w|| = 0.14691
 * Iteration #8 - Loss = 89575.190 - ||grad||^2 = 67351.91870 - ||diff_w|| = 0.05761
 * Iteration #9 - Loss = 89623.634 - ||grad||^2 = 22742.59830 - ||diff_w|| = 0.02243
 * Iteration #10 - Loss = 89626.860 - ||grad||^2 = 8637.61283 - ||diff_w|| = 0.00900
 * Iteration #11 - Loss = 89627.748 - ||grad||^2 = 3470.47562 - ||diff_w|| = 0.00378
 *

LinearSVMModel

In [11]:
c_index = concordance_index(svm_model, X_test, T_test, E_test) #0.93
print('C-index: {:f}'.format(c_index))

C-index: 0.832670


In [12]:
from pysurvival.utils import save_model

In [13]:
save_model(svm_model,'/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/SSVM/SSVM_new(onehot)_tgan-org+imbalnonar-46616.zip')

Saving the model to disk as /content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/SSVM/SSVM_new(onehot)_tgan-org+imbalnonar-46616.zip


/usr/local/lib/python3.7/dist-packages/pysurvival/utils/__init__.py:132: FutureWarning: 'pyarrow.serialize' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  model.save(path_file)


# orig+tgan-aug-51616

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from pysurvival.models.svm import LinearSVMModel
from pysurvival.models.simulations import SimulationModel
from pysurvival.utils.metrics import concordance_index
from sklearn.model_selection import train_test_split
from scipy.stats.stats import pearsonr

In [2]:
train_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/result data/for-survival-analysis/Original+TGAN-aug/orig-TGANaug-51616.csv')
test_df = pd.read_csv('/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/data/8. test_data_73.csv')

In [3]:
#train smoker df one hot encoding
Smoker_df = train_df['Smoker']

from sklearn.preprocessing import OneHotEncoder
Smoker_df= np.array(Smoker_df).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df)
ohe_encoded = ohe.transform(Smoker_df)

smoker_df2= pd.DataFrame(ohe_encoded)

new_train_df = pd.concat([train_df, smoker_df2], axis=1)
new_train_df=new_train_df.drop(['Smoker'], axis=1)

new_train_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [4]:
#test dataset one-hot-encoding
Smoker_df_test = test_df['Smoker']


Smoker_df_test= np.array(Smoker_df_test).reshape(-1, 1)
ohe = OneHotEncoder(sparse=False)
ohe.fit(Smoker_df_test)
ohe_encoded = ohe.transform(Smoker_df_test)

new_smoker_df_test= pd.DataFrame(ohe_encoded)

new_test_df = pd.concat([test_df, new_smoker_df_test], axis=1)

new_test_df=new_test_df.drop(['Smoker'], axis=1)

new_test_df.columns =['a_year_time','a_year_event','gender','age','HEIGHT','WEIGHT','PackYear','ECOG','FVC','FEV1','DLCO',
                       'DLCO_PERCENT','SCell','Aden','LCell','Positive_EGFR_Mutation','Positive_ALK_IHC','Positive_ALK_FISH',
                       'STAGE_simple','OP_Curative','RT','Chemo_curative','Chemo_Palliative','Smoker1', 'Smoker2', 'Smoker3']

In [5]:
data_train = new_train_df
data_test = new_test_df

In [6]:
sim = SimulationModel( survival_distribution = 'exponential',
                       risk_type = 'linear',
                       censored_parameter = 1,
                       alpha = 3)

In [7]:
features = [f for f in data_train.columns if f not in ['a_year_time', 'a_year_event']]

time_column = 'a_year_time'
event_column = 'a_year_event'

In [8]:
X_train, X_test = data_train[features], data_test[features]
T_train, T_test = data_train['a_year_time'].values, data_test['a_year_time'].values
E_train, E_test = data_train['a_year_event'].values, data_test['a_year_event'].values

In [9]:
from sklearn.preprocessing import StandardScaler

X_scaler = StandardScaler().fit(X_train)
X_train = X_scaler.transform(X_train)
X_test = X_scaler.transform(X_test)

In [10]:
svm_model = LinearSVMModel()
svm_model.fit(X_train, T_train, E_train, init_method='he_uniform',
    with_bias = True, lr = 0.5,  tol = 1e-3,  l2_reg = 1e-3)

Performing Newton-Raphson optimization: 
 * Iteration #1 - Loss = 10979665.926 - ||grad||^2 = 14575378.58158 - ||diff_w|| = 17.62634
 * Iteration #2 - Loss = 2815585.369 - ||grad||^2 = 7428780.43163 - ||diff_w|| = 7.90143
 * Iteration #3 - Loss = 896015.125 - ||grad||^2 = 4191577.48229 - ||diff_w|| = 4.26512
 * Iteration #4 - Loss = 311619.113 - ||grad||^2 = 2300436.78093 - ||diff_w|| = 2.09840
 * Iteration #5 - Loss = 147477.649 - ||grad||^2 = 1086431.11265 - ||diff_w|| = 0.78616
 * Iteration #6 - Loss = 113727.088 - ||grad||^2 = 509064.71934 - ||diff_w|| = 0.29305
 * Iteration #7 - Loss = 109546.777 - ||grad||^2 = 209133.68839 - ||diff_w|| = 0.11998
 * Iteration #8 - Loss = 109611.638 - ||grad||^2 = 76989.62238 - ||diff_w|| = 0.05204
 * Iteration #9 - Loss = 109703.213 - ||grad||^2 = 29764.21341 - ||diff_w|| = 0.02379
 * Iteration #10 - Loss = 109728.047 - ||grad||^2 = 12239.17638 - ||diff_w|| = 0.01164
 * Iteration #11 - Loss = 109734.555 - ||grad||^2 = 5321.25332 - ||diff_w|| = 0.0

LinearSVMModel

In [11]:
c_index = concordance_index(svm_model, X_test, T_test, E_test) #0.93
print('C-index: {:f}'.format(c_index))

C-index: 0.831115


In [12]:
from pysurvival.utils import save_model

In [13]:
save_model(svm_model,'/content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/SSVM/SSVM_new(onehot)_tgan-org+imbalnonar-51616.zip')

Saving the model to disk as /content/drive/MyDrive/PAPER/진행연구/tabular deep learning/폐암병기/modeling/SSVM/SSVM_new(onehot)_tgan-org+imbalnonar-51616.zip


/usr/local/lib/python3.7/dist-packages/pysurvival/utils/__init__.py:132: FutureWarning: 'pyarrow.serialize' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  model.save(path_file)
